In [ ]:

from __future__ import print_function, division
import downhill
from sklearn.neighbors import KernelDensity
import scipy.spatial
import numpy
from warnings import warn
from scipy.stats import scoreatpercentile
from sklearn.metrics import roc_curve, auc
from sklearn import svm
from sklearn.neighbors.base import NeighborsBase
from sklearn.neighbors.base import KNeighborsMixin
from sklearn.neighbors.base import UnsupervisedMixin
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array
import theano
import theano.tensor as T
from __future__ import print_function

__docformat__ = 'restructedtext en'

In [ ]:

"""
This tutorial introduces the multilayer perceptron using Theano.

 A multilayer perceptron is a logistic regressor where
instead of feeding the input to the logistic regression you insert a
intermediate layer, called the hidden layer, that has a nonlinear
activation function (usually tanh or sigmoid) . One can use many such
hidden layers making the architecture deep. The tutorial will also tackle
the problem of MNIST digit classification.

.. math::

    f(x) = G( b^{(2)} + W^{(2)}( s( b^{(1)} + W^{(1)} x))),

References:

    - textbooks: "Pattern Recognition and Machine Learning" -
                 Christopher M. Bishop, section 5

"""




# start-snippet-1
class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out, W=None, b=None,
                 activation=None):
        """
        Typical hidden layer of a MLP: units are fully-connected and have
        sigmoidal activation function. Weight matrix W is of shape (n_in,n_out)
        and the bias vector b is of shape (n_out,).

        NOTE : The nonlinearity used here is tanh

        Hidden unit activation is given by: tanh(dot(input,W) + b)

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.dmatrix
        :param input: a symbolic tensor of shape (n_examples, n_in)

        :type n_in: int
        :param n_in: dimensionality of input

        :type n_out: int
        :param n_out: number of hidden units

        :type activation: theano.Op or function
        :param activation: Non linearity to be applied in the hidden
                           layer
        """
        self.input = input
        # end-snippet-1

        # `W` is initialized with `W_values` which is uniformely sampled
        # from sqrt(-6./(n_in+n_hidden)) and sqrt(6./(n_in+n_hidden))
        # for tanh activation function
        # the output of uniform if converted using asarray to dtype
        # theano.config.floatX so that the code is runable on GPU
        # Note : optimal initialization of weights is dependent on the
        #        activation function used (among other things).
        #        For example, results presented in [Xavier10] suggest that you
        #        should use 4 times larger initial weights for sigmoid
        #        compared to tanh
        #        We have no info for other function, so we use the same as
        #        tanh.
        if W is None:
            W_values = numpy.asarray(
                rng.uniform(
                    low=-numpy.sqrt(6. / (n_in + n_out)),
                    high=numpy.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)
                ),
                dtype=theano.config.floatX
            )
            if activation == theano.tensor.nnet.sigmoid:
                W_values *= 4

            W = theano.shared(value=W_values, name='W', borrow=True)

        if b is None:
            b_values = numpy.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b', borrow=True)

        self.W = W
        self.b = b

        lin_output = T.dot(input, self.W) + self.b
        self.output = (lin_output if activation is None
                                    else activation(lin_output))

        # parameters of the model
        self.params = [self.W, self.b]



# start-snippet-2
#class MLP(object):
#    """Multi-Layer Perceptron Class
#
#    A multilayer perceptron is a feedforward artificial neural network model
#    that has one layer or more of hidden units and nonlinear activations.
#    Intermediate layers usually have as activation function tanh or the
#    sigmoid function (defined here by a ``HiddenLayer`` class)  while the
#    top layer is a softmax layer (defined here by a ``LogisticRegression``
#    class).
#    """
#
#    def __init__(self, rng, input, n_in, n_hidden, n_out):
#        """Initialize the parameters for the multilayer perceptron
#
#        :type rng: numpy.random.RandomState
#        :param rng: a random number generator used to initialize weights
#
#        :type input: theano.tensor.TensorType
#        :param input: symbolic variable that describes the input of the
#        architecture (one minibatch)
#
#        :type n_in: int
#        :param n_in: number of input units, the dimension of the space in
#        which the datapoints lie
#
#        :type n_hidden: int
#        :param n_hidden: number of hidden units
#
#        :type n_out: int
#        :param n_out: number of output units, the dimension of the space in
#        which the labels lie
#
#        """
#
#        # Since we are dealing with a one hidden layer MLP, this will translate
#        # into a HiddenLayer with a tanh activation function connected to the
#        # LogisticRegression layer; the activation function can be replaced by
#        # sigmoid or any other nonlinear function
#        self.hiddenLayer = HiddenLayer(
#            rng=rng,
#            input=input,
#            n_in=n_in,
#            n_out=n_hidden,
#            activation=T.tanh
#        )
#
#        # The logistic regression layer gets as input the hidden units
#        # of the hidden layer
#        self.logRegressionLayer = LogisticRegression(
#            input=self.hiddenLayer.output,
#            n_in=n_hidden,
#            n_out=n_out
#        )
#        # end-snippet-2 start-snippet-3
#        # L1 norm ; one regularization option is to enforce L1 norm to
#        # be small
#        self.L1 = (
#            abs(self.hiddenLayer.W).sum()
#            + abs(self.logRegressionLayer.W).sum()
#        )
#
#        # square of L2 norm ; one regularization option is to enforce
#        # square of L2 norm to be small
#        self.L2_sqr = (
#            (self.hiddenLayer.W ** 2).sum()
#            + (self.logRegressionLayer.W ** 2).sum()
#        )
#
#        # negative log likelihood of the MLP is given by the negative
#        # log likelihood of the output of the model, computed in the
#        # logistic regression layer
#        self.negative_log_likelihood = (
#            self.logRegressionLayer.negative_log_likelihood
#        )
#        # same holds for the function computing the number of errors
#        self.errors = self.logRegressionLayer.errors
#
#        # the parameters of the model are the parameters of the two layer it is
#        # made out of
#        self.params = self.hiddenLayer.params + self.logRegressionLayer.params
#        # end-snippet-3
#
#        # keep track of model input
#        self.input = input

#
#def test_mlp(learning_rate=0.01, L1_reg=0.00, L2_reg=0.0001, n_epochs=1000,
#             dataset='mnist.pkl.gz', batch_size=20, n_hidden=500):
#    """
#    Demonstrate stochastic gradient descent optimization for a multilayer
#    perceptron
#
#    This is demonstrated on MNIST.
#
#    :type learning_rate: float
#    :param learning_rate: learning rate used (factor for the stochastic
#    gradient
#
#    :type L1_reg: float
#    :param L1_reg: L1-norm's weight when added to the cost (see
#    regularization)
#
#    :type L2_reg: float
#    :param L2_reg: L2-norm's weight when added to the cost (see
#    regularization)
#
#    :type n_epochs: int
#    :param n_epochs: maximal number of epochs to run the optimizer
#
#    :type dataset: string
#    :param dataset: the path of the MNIST dataset file from
#                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz
#
#
#   """
#    datasets = load_data(dataset)
#
#    train_set_x, train_set_y = datasets[0]
#    valid_set_x, valid_set_y = datasets[1]
#    test_set_x, test_set_y = datasets[2]
#
#    # compute number of minibatches for training, validation and testing
#    n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
#    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] // batch_size
#    n_test_batches = test_set_x.get_value(borrow=True).shape[0] // batch_size
#
#    ######################
#    # BUILD ACTUAL MODEL #
#    ######################
#    print('... building the model')
#
#    # allocate symbolic variables for the data
#    index = T.lscalar()  # index to a [mini]batch
#    x = T.matrix('x')  # the data is presented as rasterized images
#    y = T.ivector('y')  # the labels are presented as 1D vector of
#                        # [int] labels
#
#    rng = numpy.random.RandomState(1234)
#
#    # construct the MLP class
#    classifier = MLP(
#        rng=rng,
#        input=x,
#        n_in=28 * 28,
#        n_hidden=n_hidden,
#        n_out=10
#    )
#
#    # start-snippet-4
#    # the cost we minimize during training is the negative log likelihood of
#    # the model plus the regularization terms (L1 and L2); cost is expressed
#    # here symbolically
#    cost = (
#        classifier.negative_log_likelihood(y)
#        + L1_reg * classifier.L1
#        + L2_reg * classifier.L2_sqr
#    )
#    # end-snippet-4
#
#    # compiling a Theano function that computes the mistakes that are made
#    # by the model on a minibatch
#    test_model = theano.function(
#        inputs=[index],
#        outputs=classifier.errors(y),
#        givens={
#            x: test_set_x[index * batch_size:(index + 1) * batch_size],
#            y: test_set_y[index * batch_size:(index + 1) * batch_size]
#        }
#    )
#
#    validate_model = theano.function(
#        inputs=[index],
#        outputs=classifier.errors(y),
#        givens={
#            x: valid_set_x[index * batch_size:(index + 1) * batch_size],
#            y: valid_set_y[index * batch_size:(index + 1) * batch_size]
#        }
#    )
#
#    # start-snippet-5
#    # compute the gradient of cost with respect to theta (sorted in params)
#    # the resulting gradients will be stored in a list gparams
#    gparams = [T.grad(cost, param) for param in classifier.params]
#
#    # specify how to update the parameters of the model as a list of
#    # (variable, update expression) pairs
#
#    # given two lists of the same length, A = [a1, a2, a3, a4] and
#    # B = [b1, b2, b3, b4], zip generates a list C of same size, where each
#    # element is a pair formed from the two lists :
#    #    C = [(a1, b1), (a2, b2), (a3, b3), (a4, b4)]
#    updates = [
#        (param, param - learning_rate * gparam)
#        for param, gparam in zip(classifier.params, gparams)
#    ]
#
#    # compiling a Theano function `train_model` that returns the cost, but
#    # in the same time updates the parameter of the model based on the rules
#    # defined in `updates`
#    train_model = theano.function(
#        inputs=[index],
#        outputs=cost,
#        updates=updates,
#        givens={
#            x: train_set_x[index * batch_size: (index + 1) * batch_size],
#            y: train_set_y[index * batch_size: (index + 1) * batch_size]
#        }
#    )
#    # end-snippet-5
#
#    ###############
#    # TRAIN MODEL #
#    ###############
#    print('... training')
#
#    # early-stopping parameters
#    patience = 10000  # look as this many examples regardless
#    patience_increase = 2  # wait this much longer when a new best is
#                           # found
#    improvement_threshold = 0.995  # a relative improvement of this much is
#                                   # considered significant
#    validation_frequency = min(n_train_batches, patience // 2)
#                                  # go through this many
#                                  # minibatche before checking the network
#                                  # on the validation set; in this case we
#                                  # check every epoch
#
#    best_validation_loss = numpy.inf
#    best_iter = 0
#    test_score = 0.
#    start_time = timeit.default_timer()
#
#    epoch = 0
#    done_looping = False
#
#    while (epoch < n_epochs) and (not done_looping):
#        epoch = epoch + 1
#        for minibatch_index in range(n_train_batches):
#
#            minibatch_avg_cost = train_model(minibatch_index)
#            # iteration number
#            iter = (epoch - 1) * n_train_batches + minibatch_index
#
#            if (iter + 1) % validation_frequency == 0:
#                # compute zero-one loss on validation set
#                validation_losses = [validate_model(i) for i
#                                     in range(n_valid_batches)]
#                this_validation_loss = numpy.mean(validation_losses)
#
#                print(
#                    'epoch %i, minibatch %i/%i, validation error %f %%' %
#                    (
#                        epoch,
#                        minibatch_index + 1,
#                        n_train_batches,
#                        this_validation_loss * 100.
#                    )
#                )
#
#                # if we got the best validation score until now
#                if this_validation_loss < best_validation_loss:
#                    #improve patience if loss improvement is good enough
#                    if (
#                        this_validation_loss < best_validation_loss *
#                        improvement_threshold
#                    ):
#                        patience = max(patience, iter * patience_increase)
#
#                    best_validation_loss = this_validation_loss
#                    best_iter = iter
#
#                    # test it on the test set
#                    test_losses = [test_model(i) for i
#                                   in range(n_test_batches)]
#                    test_score = numpy.mean(test_losses)
#
#                    print(('     epoch %i, minibatch %i/%i, test error of '
#                           'best model %f %%') %
#                          (epoch, minibatch_index + 1, n_train_batches,
#                           test_score * 100.))
#
#            if patience <= iter:
#                done_looping = True
#                break
#
#    end_time = timeit.default_timer()
#    print(('Optimization complete. Best validation score of %f %% '
#           'obtained at iteration %i, with test performance %f %%') %
#          (best_validation_loss * 100., best_iter + 1, test_score * 100.))
#    print(('The code for file ' +
#           os.path.split(__file__)[1] +
#           ' ran for %.2fm' % ((end_time - start_time) / 60.)), file=sys.stderr)
#
#
#if __name__ == '__main__':
#    test_mlp()


In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Thu Aug 17 15:26:18 2017

@author: VANLOI
"""



"calculate batch_size on different size of dataset"
def stopping_para_vae(train_size):
    n_batch  = 0
    b_size = 0
    max_stop = 2000
    if (train_size <= 2000):
        n_batch  = 20
        b_size = int(train_size/n_batch)
    else:
        b_size = 100
        n_batch  = int(train_size/b_size)

    patience = round(max_stop/n_batch)
    every_valid = 1
    return patience, every_valid, b_size, n_batch


def stopping_para_shrink(train_size):
    n_batch  = 0
    b_size = 0
    max_stop = 2000.0
    if (train_size <= 2000):
        n_batch  = 20
        b_size = int(train_size/n_batch)
    else:
        b_size = 100
        n_batch  = int(train_size/b_size)

    every_valid = 5
    "update times after validation = every_valid x n_batch"
    patience =  round(max_stop/(every_valid*n_batch))

    return patience, every_valid, b_size, n_batch

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Thu Apr 06 12:59:05 2017

@author: VANLOI
"""
NSLKDD = ["Probe", "DoS", "R2L", "U2R", "NSLKDD"]
UNSW   = ["Fuzzers", "Analysis", "Backdoor", "DoS_UNSW", "Exploits", "Generic",\
            "Reconnaissance", "Shellcode", "Worms", "UNSW"]
CTU13  = ["CTU13_08", "CTU13_13", "CTU13_10", "CTU13_09"]

def hyper_parameters(data):
    if (data == "C-heart"):
        h_sizes = [10, 7, 4]

    elif (data == "ACA"):
        h_sizes = [11, 7, 4]

    elif (data == "WBC"):
        h_sizes = [7, 6, 4]

    elif (data == "WDBC"):
        h_sizes = [22, 14, 6]

    elif (data in NSLKDD):
        h_sizes = [85, 49, 12]

    elif (data in UNSW):
        h_sizes = [136, 75, 15]

    #Table - 1
    elif (data == "GLASS"):
        h_sizes = [7, 6, 4]

    elif (data == "Ionosphere"):
        h_sizes = [23, 15, 6]


    elif (data == "PenDigits"):
        h_sizes = [12, 9, 5]

    elif (data == "Shuttle"):
        h_sizes = [7, 6, 4]

    elif (data == "WPBC"):
        h_sizes = [23, 15, 6]

    #Table - 2
    elif (data == "Annthyroid"):
        h_sizes = [16, 10, 5]

    elif (data == "Arrhythmia"):
        h_sizes = [178, 98, 17]
        #h_sizes = [138, 17]
        #h_sizes = [50, 10, 2]

    elif (data == "Cardiotocography"):
        h_sizes = [16, 10, 5]

    elif (data == "Heartdisease"):
        h_sizes = [10, 7, 4]

    elif (data == "Hepatitis"):
        h_sizes = [14, 10, 5]

    elif (data == "InternetAds"):
        h_sizes = [1052, 546, 40]

    elif (data == "PageBlocks"):
        h_sizes = [8, 6, 4]

    elif (data == "Parkinson"):
        h_sizes = [16, 11, 5]

    elif (data == "Pima"):
        h_sizes = [6, 5, 3]

    elif (data == "Spambase"):
        h_sizes = [41, 24, 8]

    elif (data == "Wilt"):
        h_sizes = [4, 4, 3]


    elif (data == "waveform"):
        h_sizes = [14, 7, 5]

    elif (data == "CTU13_08"):
        h_sizes = [29, 18, 7]
        #h_sizes = [27, 15, 2]

    elif (data == "CTU13_09"):
        h_sizes = [30, 18, 7]
        #h_sizes = [28, 15, 2]

    elif (data == "CTU13_10"):
        h_sizes = [28, 17, 7]
        #h_sizes = [26, 14, 2]

    elif (data == "CTU13_13"):
        h_sizes = [29, 18, 7]
        #h_sizes = [27, 15, 2]

    elif (data == "CTU13_06"):
        h_sizes = [28, 17, 7]

    elif (data == "CTU13_07"):
        h_sizes = [25, 15, 6]

    elif (data == "CTU13_12"):
        h_sizes = [26, 17, 7]

    return h_sizes

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Mon Mar 06 17:29:51 2017

@author: VANLOI
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Mar 06 17:16:02 2017

@author: VANLOI
"""



__all__ = ["LocalOutlierFactor"]


class LocalOutlierFactor(NeighborsBase, KNeighborsMixin, UnsupervisedMixin):
    """Unsupervised Outlier Detection using Local Outlier Factor (LOF)
    The anomaly score of each sample is called Local Outlier Factor.
    It measures the local deviation of density of a given sample with
    respect to its neighbors.
    It is local in that the anomaly score depends on how isolated the object
    is with respect to the surrounding neighborhood.
    More precisely, locality is given by k-nearest neighbors, whose distance
    is used to estimate the local density.
    By comparing the local density of a sample to the local densities of
    its neighbors, one can identify samples that have a substantially lower
    density than their neighbors. These are considered outliers.
    Parameters
    ----------
    n_neighbors : int, optional (default=20)
        Number of neighbors to use by default for :meth:`kneighbors` queries.
        If n_neighbors is larger than the number of samples provided,
        all samples will be used.
    algorithm : {'auto', 'ball_tree', 'kd_tree', 'brute'}, optional
        Algorithm used to compute the nearest neighbors:
        - 'ball_tree' will use :class:`BallTree`
        - 'kd_tree' will use :class:`KDTree`
        - 'brute' will use a brute-force search.
        - 'auto' will attempt to decide the most appropriate algorithm
          based on the values passed to :meth:`fit` method.
        Note: fitting on sparse input will override the setting of
        this parameter, using brute force.
    leaf_size : int, optional (default=30)
        Leaf size passed to :class:`BallTree` or :class:`KDTree`. This can
        affect the speed of the construction and query, as well as the memory
        required to store the tree. The optimal value depends on the
        nature of the problem.
    p : integer, optional (default=2)
        Parameter for the Minkowski metric from
        :ref:`sklearn.metrics.pairwise.pairwise_distances`. When p = 1, this is
        equivalent to using manhattan_distance (l1), and euclidean_distance
        (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.
    metric : string or callable, default 'minkowski'
        metric used for the distance computation. Any metric from scikit-learn
        or scipy.spatial.distance can be used.
        If 'precomputed', the training input X is expected to be a distance
        matrix.
        If metric is a callable function, it is called on each
        pair of instances (rows) and the resulting value recorded. The callable
        should take two arrays as input and return one value indicating the
        distance between them. This works for Scipy's metrics, but is less
        efficient than passing the metric name as a string.
        Valid values for metric are:
        - from scikit-learn: ['cityblock', 'cosine', 'euclidean', 'l1', 'l2',
          'manhattan']
        - from scipy.spatial.distance: ['braycurtis', 'canberra', 'chebyshev',
          'correlation', 'dice', 'hamming', 'jaccard', 'kulsinski',
          'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto',
          'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath',
          'sqeuclidean', 'yule']
        See the documentation for scipy.spatial.distance for details on these
        metrics:
        http://docs.scipy.org/doc/scipy/reference/spatial.distance.html
    metric_params : dict, optional (default=None)
        Additional keyword arguments for the metric function.
    contamination : float in (0., 0.5), optional (default=0.1)
        The amount of contamination of the data set, i.e. the proportion
        of outliers in the data set. When fitting this is used to define the
        threshold on the decision function.
    n_jobs : int, optional (default=1)
        The number of parallel jobs to run for neighbors search.
        If ``-1``, then the number of jobs is set to the number of CPU cores.
        Affects only :meth:`kneighbors` and :meth:`kneighbors_graph` methods.
    Attributes
    ----------
    negative_outlier_factor_ : numpy array, shape (n_samples,)
        The opposite LOF of the training samples. The lower, the more normal.
        Inliers tend to have a LOF score close to 1, while outliers tend
        to have a larger LOF score.
        The local outlier factor (LOF) of a sample captures its
        supposed 'degree of abnormality'.
        It is the average of the ratio of the local reachability density of
        a sample and those of its k-nearest neighbors.
    n_neighbors_ : integer
        The actual number of neighbors used for :meth:`kneighbors` queries.
    References
    ----------
    .. [1] Breunig, M. M., Kriegel, H. P., Ng, R. T., & Sander, J. (2000, May).
           LOF: identifying density-based local outliers. In ACM sigmod record.
    """
    def __init__(self, n_neighbors=20, algorithm='auto', leaf_size=30,
                 metric='minkowski', p=2, metric_params=None,
                 contamination=0.1, n_jobs=1):
                    self.n_neighbors=n_neighbors
                    self.algorithm=algorithm
                    self.leaf_size=leaf_size
                    self.metric=metric
                    self.p=p
                    self.metric_params=metric_params
                    self.n_jobs=n_jobs

                    self.contamination = contamination

    def fit_predict(self, X, y=None):
        """"Fits the model to the training set X and returns the labels
        (1 inlier, -1 outlier) on the training set according to the LOF score
        and the contamination parameter.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features), default=None
            The query sample or samples to compute the Local Outlier Factor
            w.r.t. to the training samples.
        Returns
        -------
        is_inlier : array, shape (n_samples,)
            Returns -1 for anomalies/outliers and 1 for inliers.
        """

        return self.fit(X)._predict()

    def fit(self, X, y=None):
        """Fit the model using X as training data.
        Parameters
        ----------
        X : {array-like, sparse matrix, BallTree, KDTree}
            Training data. If array or matrix, shape [n_samples, n_features],
            or [n_samples, n_samples] if metric='precomputed'.
        Returns
        -------
        self : object
            Returns self.
        """
        if not (0. < self.contamination <= .5):
            raise ValueError("contamination must be in (0, 0.5]")

        super(LocalOutlierFactor, self).fit(X)

        n_samples = self._fit_X.shape[0]
        if self.n_neighbors > n_samples:
            warn("n_neighbors (%s) is greater than the "
                 "total number of samples (%s). n_neighbors "
                 "will be set to (n_samples - 1) for estimation."
                 % (self.n_neighbors, n_samples))
        self.n_neighbors_ = max(1, min(self.n_neighbors, n_samples - 1))

        self._distances_fit_X_, _neighbors_indices_fit_X_ = (
            self.kneighbors(None, n_neighbors=self.n_neighbors_))

        self._lrd = self._local_reachability_density(
            self._distances_fit_X_, _neighbors_indices_fit_X_)

        # Compute lof score over training samples to define threshold_:
        lrd_ratios_array = (self._lrd[_neighbors_indices_fit_X_] /
                            self._lrd[:, np.newaxis])

        self.negative_outlier_factor_ = -np.mean(lrd_ratios_array, axis=1)

        self.threshold_ = -scoreatpercentile(
            -self.negative_outlier_factor_, 100. * (1. - self.contamination))

        return self

    def _predict(self, X=None):
        """Predict the labels (1 inlier, -1 outlier) of X according to LOF.
        If X is None, returns the same as fit_predict(X_train).
        This method allows to generalize prediction to new observations (not
        in the training set). As LOF originally does not deal with new data,
        this method is kept private.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features), default=None
            The query sample or samples to compute the Local Outlier Factor
            w.r.t. to the training samples. If None, makes prediction on the
            training data without considering them as their own neighbors.
        Returns
        -------
        is_inlier : array, shape (n_samples,)
            Returns -1 for anomalies/outliers and +1 for inliers.
        """
        check_is_fitted(self, ["threshold_", "negative_outlier_factor_",
                               "n_neighbors_", "_distances_fit_X_"])

        if X is not None:
            X = check_array(X, accept_sparse='csr')
            is_inlier = np.ones(X.shape[0], dtype=int)
            is_inlier[self._decision_function(X) <= self.threshold_] = -1
        else:
            is_inlier = np.ones(self._fit_X.shape[0], dtype=int)
            is_inlier[self.negative_outlier_factor_ <= self.threshold_] = -1

        return is_inlier

    def _decision_function(self, X):
        """Opposite of the Local Outlier Factor of X (as bigger is better,
        i.e. large values correspond to inliers).
        The argument X is supposed to contain *new data*: if X contains a
        point from training, it consider the later in its own neighborhood.
        Also, the samples in X are not considered in the neighborhood of any
        point.
        The decision function on training data is available by considering the
        opposite of the negative_outlier_factor_ attribute.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The query sample or samples to compute the Local Outlier Factor
            w.r.t. the training samples.
        Returns
        -------
        opposite_lof_scores : array, shape (n_samples,)
            The opposite of the Local Outlier Factor of each input samples.
            The lower, the more abnormal.
        """
        check_is_fitted(self, ["threshold_", "negative_outlier_factor_",
                               "_distances_fit_X_"])

        X = check_array(X, accept_sparse='csr')

        distances_X, neighbors_indices_X = (
            self.kneighbors(X, n_neighbors=self.n_neighbors_))
        X_lrd = self._local_reachability_density(distances_X,
                                                 neighbors_indices_X)

        lrd_ratios_array = (self._lrd[neighbors_indices_X] /
                            X_lrd[:, np.newaxis])

        # as bigger is better:
        return -np.mean(lrd_ratios_array, axis=1)

    def _local_reachability_density(self, distances_X, neighbors_indices):
        """The local reachability density (LRD)
        The LRD of a sample is the inverse of the average reachability
        distance of its k-nearest neighbors.
        Parameters
        ----------
        distances_X : array, shape (n_query, self.n_neighbors)
            Distances to the neighbors (in the training samples `self._fit_X`)
            of each query point to compute the LRD.
        neighbors_indices : array, shape (n_query, self.n_neighbors)
            Neighbors indices (of each query point) among training samples
            self._fit_X.
        Returns
        -------
        local_reachability_density : array, shape (n_samples,)
            The local reachability density of each sample.
        """
        dist_k = self._distances_fit_X_[neighbors_indices,
                                        self.n_neighbors_ - 1]
        reach_dist_array = np.maximum(distances_X, dist_k)

        #  1e-10 to avoid `nan' when when nb of duplicates > n_neighbors_:
        return 1. / (np.mean(reach_dist_array, axis=1) + 1e-10)

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Thu Feb 25 17:47:44 2016

@author: caoloi
"""
import numpy as np
from sklearn import preprocessing

seed = 0

"Normalize training and testing sets"
def normalize_data(train_X, test_X, scale):
    if ((scale == "standard") | (scale == "maxabs") | (scale == "minmax")):
        if (scale == "standard"):
            scaler = preprocessing.StandardScaler()
        elif (scale == "maxabs"):
            scaler = preprocessing.MaxAbsScaler()
        elif (scale == "minmax"):
            scaler = preprocessing.MinMaxScaler()
        scaler.fit(train_X)
        train_X = scaler.transform(train_X)
        test_X  = scaler.transform(test_X)
    else:
        print ("No scaler")
    return train_X, test_X

"***********************Pre-processing NSL-KDD dataset************************"
def process_KDD(group_attack):
    d0 = np.genfromtxt("Data/NSLKDD/NSLKDD_Train.csv", delimiter=",")
    d1 = np.genfromtxt("Data/NSLKDD/NSLKDD_Test.csv", delimiter=",")
    # Train: 67343, Test normal: 9711, Dos: 7458, R2L   2887, U2R: 67, Probe 2422

    n_train = 6734

    np.random.seed(seed)
    np.random.shuffle(d0)
    np.random.shuffle(d1)

    "Pre-processing training set"
    dy = d0[:,-1]                       # put labels(the last column) to dy
    train_X = d0[(dy==0)]               # Normal(class 0) to train_X
    train_X = train_X[:,0:-1]           # discard the last column (labels)
    train_X = train_X[:n_train]         # downsample training set

    "Pre-processing Testing set"
    dy = d1[:,-1]                       # put labels to dy
    dX = d1[:,0:-1]                     # put data to dX without last column (labels)

    dX0 = dX[(dy==0)]                   # Normal, class 0
    if (group_attack == "Probe"):
        dX1 = dX[(dy==1)]
    elif (group_attack == "DoS"):
        dX1 = dX[(dy==2)]
    elif (group_attack == "R2L"):
        dX1 = dX[(dy==3)]
    elif (group_attack == "U2R"):
        dX1 = dX[(dy==4)]
    elif (group_attack == "NSLKDD"):
        dX1 = dX[(dy>0)]
    else:
        print ("No group of attack")

    test_X0 = dX0                       #normal test
    test_X1 = dX1                       #anomaly test
    #normal and anomaly test
    test_X = np.concatenate((test_X0, test_X1))

    #Create label for normal and anomaly test examples, and then combine two sets
    test_y0 = np.full((len(test_X0)), False, dtype=bool)
    test_y1 = np.full((len(test_X1)), True,  dtype=bool)
    test_y =  np.concatenate((test_y0, test_y1))
    #create binary label (1-normal, 0-anomaly) for compute AUC later
    actual = (~test_y).astype(np.int)

    return train_X, test_X, actual

"***************************Processing UNSW Dataset***************************"
def process_UNSW(group_attack):

    """group_attack: {'Fuzzers': 1, 'Exploits': 5, 'Normal': 0, 'Generic': 6, 'Worms': 9,
    'Analysis': 2, 'Backdoor': 3, 'DoS': 4, 'Reconnaissance': 7, 'Shellcode': 8}"""
    #DoS --> DoS_UNSW to avoid the conflict with DoS in NSL-KDD
    d0 = np.genfromtxt("Data/UNSW/UNSW_Train.csv", delimiter=",")
    d1 = np.genfromtxt("Data/UNSW/UNSW_Test.csv", delimiter=",")
    #Training set: 175341 (56000 normal, 119341 anomaly)
    #Testing set:   82332 (37000 normal,  45332 anomaly)

    n_train = 5600         #downsample training set 10%

    np.random.seed(seed)
    np.random.shuffle(d0)
    np.random.shuffle(d1)

    "Pre-processing training set"
    dy = d0[:,-1]                       # put labels(the last column) to dy
    train_X = d0[(dy==0)]               # Normal(class 0) to train_X
    train_X = train_X[:,0:-1]           # discard the last column (labels)
    train_X = train_X[:n_train]         # downsample training set

    "Pre-processing Testing set"
    dy = d1[:,-1]                       # put labels to dy
    dX = d1[:,0:-1]                     # put data to dX without last column (labels)


    """group_attack: {'Fuzzers': 1, 'Exploits': 5, 'Normal': 0, 'Generic': 6, 'Worms': 9,
    'Analysis': 2, 'Backdoor': 3, 'DoS': 4, 'Reconnaissance': 7, 'Shellcode': 8}"""

    dX0 = dX[(dy==0)]                   # Normal, class 0

    if (group_attack == "Fuzzers"):
        dX1 = dX[(dy==1)]
    elif (group_attack == "Analysis"):
        dX1 = dX[(dy==2)]
    elif (group_attack == "Backdoor"):
        dX1 = dX[(dy==3)]
    elif (group_attack == "DoS_UNSW"):
        dX1 = dX[(dy==4)]
    elif (group_attack == "Exploits"):
        dX1 = dX[(dy==5)]
    elif (group_attack == "Generic"):
        dX1 = dX[(dy==6)]
    elif (group_attack == "Reconnaissance"):
        dX1 = dX[(dy==7)]
    elif (group_attack == "Shellcode"):
        dX1 = dX[(dy==8)]
    elif (group_attack == "Worms"):
        dX1 = dX[(dy==9)]
    elif (group_attack == "UNSW"):
        dX1 = dX[(dy>0)]
    else:
        print ("No group of attack")

    test_X0 = dX0                       #normal test
    test_X1 = dX1                       #anomaly test
    #normal and anomaly test
    test_X = np.concatenate((test_X0, test_X1))

    #Create label for normal and anomaly test examples, and then combine two sets
    test_y0 = np.full((len(test_X0)), False, dtype=bool)
    test_y1 = np.full((len(test_X1)), True,  dtype=bool)
    test_y =  np.concatenate((test_y0, test_y1))
    #create binary label (1-normal, 0-anomaly) for compute AUC later
    actual = (~test_y).astype(np.int)

    return train_X, test_X, actual

"*************************Read data from CSV file*****************************"
def process_PenDigits():
    #16 real-values + 1 class (normal: class 0, anomaly: class 2 or 1-9)
    #each class is equaly to each others in train and test set
    d0 = np.genfromtxt("Data/pendigits_train.csv", delimiter=",")
    d1 = np.genfromtxt("Data/pendigits_test.csv", delimiter=",")

    # shuffle
    np.random.seed(seed)
    np.random.shuffle(d0)
    np.random.shuffle(d1)

    "Pre-processing training set"
    dy = d0[:,-1]                       # put labels(the last column) to dy
    train_X = d0[(dy==0)]               # Normal(class 0) to train_X
    train_X = train_X[:,0:-1]           # discard the last column (labels)

    "Pre-processing Testing set"
    dy = d1[:,-1]                       # put labels to dy
    dX = d1[:,0:-1]                     # put data to dX without last column (labels)

    dX0 = dX[(dy == 0)]                  # Normal, class 0
    dX1 = dX[(dy == 2)]                   # Anomaly, class 2 (or 1-9)

    test_X0 = dX0                       #normal test
    test_X1 = dX1                       #anomaly test
    #normal and anomaly test
    test_X = np.concatenate((test_X0, test_X1))

    #Create label for normal and anomaly test examples, and then combine two sets
    test_y0 = np.full((len(test_X0)), False, dtype=bool)
    test_y1 = np.full((len(test_X1)), True,  dtype=bool)
    test_y =  np.concatenate((test_y0, test_y1))
    #create binary label (1-normal, 0-anomaly) for compute AUC later
    actual = (~test_y).astype(np.int)

    return train_X, test_X, actual


"*************************Read Shuttle data*****************************"
def process_Shuttle():
    #9 attributes + 1 class (1-7), train_set: 43500(34108 normal), test_set: 14500
    #Consider normal: class 1, anomaly: classes 2 - 7
    d0 = np.genfromtxt("Data/shuttle_train.csv", delimiter=",")
    d1 = np.genfromtxt("Data/shuttle_test.csv", delimiter=",")
    n_train = 3410  #downsample 10%
    # shuffle
    np.random.seed(seed)
    np.random.shuffle(d0)
    np.random.shuffle(d1)

    "Pre-processing training set"
    dy = d0[:,-1]                       # put labels(the last column) to dy
    train_X = d0[(dy==1)]               # Normal(class 1) to train_X
    train_X = train_X[:,0:-1]           # discard the last column (labels)

    train_X = train_X[:n_train]

    "Pre-processing Testing set"
    dy = d1[:,-1]                       # put labels to dy
    dX = d1[:,0:-1]                     # put data to dX without last column (labels)

    dX0 = dX[(dy==1)]                   # Normal, class 1
    dX1 = dX[(dy>1)]                    # Anomaly, class 2-7

    test_X0 = dX0                       #normal test
    test_X1 = dX1                       #anomaly test
    #normal and anomaly test
    test_X = np.concatenate((test_X0, test_X1))

    #Create label for normal and anomaly test examples, and then combine two sets
    test_y0 = np.full((len(test_X0)), False, dtype=bool)
    test_y1 = np.full((len(test_X1)), True,  dtype=bool)
    test_y =  np.concatenate((test_y0, test_y1))
    actual = (~test_y).astype(np.int)

    return train_X, test_X, actual


"*************************Read Annthyroid data*****************************"
def process_Annthyroid():
    #21 attributes + 1 class (3 - healthy, 1-2 - hypothyroidism)
    #Train_set (3488 1-heathy, 284 hypo), Test_set(3178-heathy, 250-hypo)
    # 3 - normal, 1 - hyperfunction and 2 - subnormal functioning
    d0 = np.genfromtxt("Data/annthyroid_train.csv", delimiter=",")
    d1 = np.genfromtxt("Data/annthyroid_test.csv", delimiter=",")
    n_train = 3488
    # shuffle
    np.random.seed(seed)
    np.random.shuffle(d0)
    np.random.shuffle(d1)

    "Pre-processing training set"
    dy = d0[:,-1]                       # put labels(the last column) to dy
    train_X = d0[(dy==3)]               # Normal(class 3) to train_X
    train_X = train_X[:,0:-1]           # discard the last column (labels)
    train_X = train_X[:n_train]

    "Pre-processing Testing set"
    dy = d1[:,-1]                       # put labels to dy
    dX = d1[:,0:-1]                     # put data to dX without last column (labels)

    dX0 = dX[(dy==3)]                   # Normal, class 1
    dX1 = dX[(dy<3)]                    # Anomaly, class 1, 2. better if choosing only class 1

    test_X0 = dX0                       #normal test
    test_X1 = dX1                       #anomaly test
    #normal and anomaly test
    test_X = np.concatenate((test_X0, test_X1))

    #Create label for normal and anomaly test examples, and then combine two sets
    test_y0 = np.full((len(test_X0)), False, dtype=bool)
    test_y1 = np.full((len(test_X1)), True,  dtype=bool)
    test_y =  np.concatenate((test_y0, test_y1))
    actual = (~test_y).astype(np.int)

    return train_X, test_X, actual

"*************************Read wilt data*****************************"
def process_wilt():
    #5 attributes + 1 class(0,1), Train_set(4265 cover land, 74 diseased tree)
    #Test_set (313 cover land, 187 diseased tree)
    d0 = np.genfromtxt("Data/wilt_train.csv", delimiter=",")
    d1 = np.genfromtxt("Data/wilt_test.csv", delimiter=",")
    n_train = 4265
    # shuffle
    np.random.seed(seed)
    np.random.shuffle(d0)
    np.random.shuffle(d1)

    "Pre-processing training set"
    dy = d0[:,-1]                       # put labels(the last column) to dy
    train_X = d0[(dy==0)]               # Normal(class 1) to train_X
    train_X = train_X[:,0:-1]           # discard the last column (labels)
    train_X = train_X[:n_train]         #downsample\


    "Pre-processing Testing set"
    dX = d1[:,0:-1]                     # put data to dX without last column (labels)
    dy = d1[:,-1]                       # put labels to dy

    dX0 = dX[(dy==0)]                   # Normal, class 0 (cover land)
    dX1 = dX[(dy==1)]                   # Anomaly, class 1 (diseased tree)

    test_X0 = dX0                       #normal test
    test_X1 = dX1                       #anomaly test
    #normal and anomaly test
    test_X = np.concatenate((test_X0, test_X1))

    #Create label for normal and anomaly test examples, and then combine two sets
    test_y0 = np.full((len(test_X0)), False, dtype=bool)
    test_y1 = np.full((len(test_X1)), True,  dtype=bool)
    test_y =  np.concatenate((test_y0, test_y1))
    actual = (~test_y).astype(np.int)

    return train_X, test_X, actual

"*****************************Load dataset*****************************"
def load_data(dataset):
    NSLKDD = ["Probe", "DoS", "R2L", "U2R", "NSLKDD"]
    UNSW   = ["Fuzzers", "Analysis", "Backdoor", "DoS_UNSW", "Exploits", "Generic",\
            "Reconnaissance", "Shellcode", "Worms", "UNSW"]
    CTU13  = ["CTU13_06","CTU13_07","CTU13_08","CTU13_09","CTU13_10","CTU13_12","CTU13_13"]

    if (dataset == "C-heart"):
        d = np.genfromtxt("Data/C-heart.csv", delimiter=",")
        label_threshold = 0
        # 13 attributes + 1 class [0 - Level0(164); level 1,2,3,4 - (139), 6 missing)
        #Some features may be CATEGORICAL, don't need to preprocessing

    elif (dataset == "ACA"):
        d = np.genfromtxt("Data/australian.csv", delimiter=",")
        label_threshold = 0
        # 14 feature + 1 class [ 0 (383 normal), 1 (307 anomaly)]
        # 8 CATEGORICAL FEATURES NEED TO BE PREPROCESSED

    elif (dataset == "WBC"):
        d = np.genfromtxt("Data/wbc.csv", delimiter=",")
        label_threshold = 2
        #9 real-value + 1 class[2-benign(458); 4-malignant(241)], 16 missing

    elif (dataset == "WDBC"):
        d = np.genfromtxt("Data/wdbc.csv", delimiter=",")
        label_threshold = 2
        # 30 real-value + 1 class [2 - benign(357); 4 - malignant(212)]

    elif (dataset in NSLKDD):
        train_X, test_X, actual = process_KDD(dataset)
        return train_X, test_X, actual
        #Tree CATEGORICAL FEATURES NEED TO BE PREPROCESSED

    elif (dataset in UNSW):
        train_X, test_X, actual = process_UNSW(dataset)
        return train_X, test_X, actual

    #**************************** TABLE - 1 **********************************
    elif (dataset == "GLASS"):
        d = np.genfromtxt("Data/glass.csv", delimiter=",")
        label_threshold = 4
        # 9 attributes + 1 class [1-4 - 163 window glass (normal); 5-7 - 51 Non-window glass (anomaly)]

    elif (dataset == "Ionosphere"):
        d = np.genfromtxt("Data/ionosphere.csv", delimiter=",")
        label_threshold = 0
        d = d[:,2:]
        # Ignore the first and second features, using 32 features
        # 34 attributes + 1 class [0 - 225 good (normal); 1 - 126 bad (anomaly)]

    elif (dataset == "PenDigits"):
        train_X, test_X, actual = process_PenDigits()
        #16 real-value + 1 class attribute (0 as Normal - 2 ( or 1,2,3,4,5,6,7,8,9)
        #Training 780 normal, testing 363 normal and 364 anomaly
        return train_X, test_X, actual

    elif (dataset == "Shuttle"):
        train_X, test_X, actual = process_Shuttle()
        #9 real-value + 1 class (1 as Normal -  class 2-7 as anomaly)
        #train_set: 43500, test_set: 14500
        return train_X, test_X, actual

    elif (dataset == "WPBC"):
        d = np.genfromtxt("Data/wpbc.csv", delimiter=",")
        label_threshold = 0
        d = d[:,1:]   #remove ID feature
        # 32 attributes + class [0 - 151 nonrecur (normal); 1 - 47 recur (anomaly)], 4 missing


    #**************************** TABLE - 2 **********************************
    elif (dataset == "Annthyroid"):
        train_X, test_X, actual = process_Annthyroid()
        #21 attributes + 1 class(Class 3 as Normal -  class 1, 2 as anomaly)
        #Training 3488 normal, testing 3178 normal and 250 anomaly
        return train_X, test_X, actual

    elif (dataset =="Arrhythmia"):
        d = np.genfromtxt("Data/arrhythmia.csv", delimiter=",")
        # 452, 245 normal (class 1), 207 anomaly (classes 2 - 16)
        # 279 attributes, 19 attributes have value 0, 1 attribute has many missing data.
        label_threshold = 1

    elif (dataset =="Cardiotocography"):
        d = np.genfromtxt("Data/Cardiotocography.csv", delimiter=",")
        # 21, 1655 normal (class 1), 471 anomaly (classes 2, 3)
        label_threshold = 1

    elif (dataset =="Heartdisease"):
        d = np.genfromtxt("Data/heartdisease.csv", delimiter=",")
        # 270 instances: 150 absence (1-normal) and 120 presence (2-anomaly)
        #13 attributes including some: ORDERED and NOMINAL features
        label_threshold = 1

    elif (dataset =="Hepatitis"):
        d = np.genfromtxt("Data/hepatitis.csv", delimiter=",")
        # 155 instances, 19 attributes + class label( 2- Live 123, 1-die 32)
        #(remove missing: remain class 2: 67, class 1: 13)
        label_threshold = 2

    elif (dataset =="InternetAds"):
        d = np.genfromtxt("Data/internet-ad.csv", delimiter=",")
        # 3264 instances, 1558 attributes + class(0: nonad, 1: Ads), many missing
        label_threshold = 0

    elif (dataset =="PageBlocks"):
        d = np.genfromtxt("Data/page-blocks.csv", delimiter=",")
        # 5473 instances, 10 attributes + class(1 (4913): text, 2-5: hiriz line (329), graphic (28),
        # line (88) or picture (115)
        label_threshold = 1

    elif (dataset =="Parkinson"):
        d = np.genfromtxt("Data/parkinsons.csv", delimiter=",")
        # 195 instances: 48 Healthy (0), 147 Parkinson (1), 22 real-value
        label_threshold = 0

    elif (dataset =="Pima"):
        d = np.genfromtxt("Data/pima.csv", delimiter=",")
        # 768,  500 normal (class 0), 268 diabetes (class 1)
        # 8 real, integer - values attributes
        label_threshold = 0

    elif (dataset =="Spambase"):
        d = np.genfromtxt("Data/spambase.csv", delimiter=",")
        # 4601,  2788 non-spam (normal, 0), 1813 spam (anomaly, 1), 57 real-values
        label_threshold = 0

    elif (dataset == "Wilt"):
        train_X, test_X, actual = process_wilt()
        #5 attributes + 1 class attribute, (0) Non-wilt as Normal - (1) wilt as anomaly
        #Training 4388 ( 4265 - normal, 74 anomaly), testing 500 (313 - normal, 187 - anomaly)
        return train_X, test_X, actual


    elif (dataset =="CTU13_08"):
        d = np.genfromtxt("Data/CTU13/CTU13_08.csv", delimiter=",")
        label_threshold = 1

    elif (dataset =="CTU13_09"):
        d = np.genfromtxt("Data/CTU13/CTU13_09.csv", delimiter=",")
        label_threshold = 1

    elif (dataset =="CTU13_10"):
        d = np.genfromtxt("Data/CTU13/CTU13_10.csv", delimiter=",")
        label_threshold = 1

    elif (dataset =="CTU13_13"):
        d = np.genfromtxt("Data/CTU13/CTU13_13.csv", delimiter=",")
        label_threshold = 1


    elif (dataset =="CTU13_06"):
        d = np.genfromtxt("Data/CTU13/CTU13_06.csv", delimiter=",")
        label_threshold = 1

    elif (dataset =="CTU13_07"):
        d = np.genfromtxt("Data/CTU13/CTU13_07.csv", delimiter=",")
        label_threshold = 1

    elif (dataset =="CTU13_12"):
        d = np.genfromtxt("Data/CTU13/CTU13_12.csv", delimiter=",")
        label_threshold = 1

    else:
        print ("Incorrect data")

    "*************************Chosing dataset*********************************"
    d = d[~np.isnan(d).any(axis=1)]    #discard the '?' values

    np.random.seed(seed)
    np.random.shuffle(d)

    dX = d[:,0:-1]              #put data to dX without the last column (labels)
    dy = d[:,-1]                #put label to dy

    if (dataset =="Hepatitis"):
        dy = dy < label_threshold
    else:
        dy = dy > label_threshold
                                # dy=True with anomaly labels
                                # separate into normal and anomaly
    dX0 = dX[~dy]               # Normal data
    dX1 = dX[dy]                # Anomaly data
    dy0 = dy[~dy]               # Normal label
    dy1 = dy[dy]                # Anomaly label

    #print("Normal: %d Anomaly %d" %(len(dX0), len(dX1)))
    if (dataset in CTU13):
        split = 0.4             #split 40% for training, 60% for testing
    else:
        split = 0.8             #split 80% for training, 20% for testing

    idx0  = int(split * len(dX0))
    idx1  = int(split * len(dX1))

    train_X = dX0[:idx0]        # train_X is 80% of the normal class

    # test set is the other half of the normal class and all of the anomaly class
    test_X = np.concatenate((dX0[idx0:], dX1[idx1:]))  # 30% of normal and 30% of anomaly
    test_y = np.concatenate((dy0[idx0:], dy1[idx1:]))  # 30% of normal and 30% of anomaly label
    #conver test_y into 1 or 0 for computing AUC later
    actual = (~test_y).astype(np.int)

    return train_X, test_X, actual

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Thu Feb 25 18:06:19 2016

@author: caoloi
"""

"******************************* CENTROID *************************************"
class CentroidBasedOneClassClassifier:
    def __init__(self, threshold = 0.95, metric="euclidean", scale = "standard"):

        self.threshold = threshold
        """only CEN used StandardScaler because the centroid of training set need
        to be move to origin"""
        self.scaler = preprocessing.StandardScaler()
        self.metric = metric

    def fit(self, X):
        self.scaler.fit(X)
        X = self.scaler.transform(X)
        # because we are using StandardScaler, the centroid is a
        # vector of zeros, but we save it in shape (1, n) to allow
        # cdist to work happily later.
        self.centroid = np.zeros((1, X.shape[1]))
        # no need to scale again
        dists = self.get_density(X, scale=False)
        # transform relative threshold (eg 95%) to absolute
        self.abs_threshold = np.percentile(dists, 100 * self.threshold)
    #It is actually the mean of the distances from each points in training set
    #to the centroid zero.
    def get_density(self, X, scale=True):
        if scale:
            X = self.scaler.transform(X)
        dists = scipy.spatial.distance.cdist(X, self.centroid, metric=self.metric)
        dists = np.mean(dists, axis=1)
        return dists

    def predict(self, X):
        dists = self.get_density(X)
        return dists > self.abs_threshold

"****************** CENTROID WITHOUT STANDARD SCALER *************************"
class Centroid_Classifier:
    def __init__(self, threshold = 0.95, metric="euclidean"):

        self.threshold = threshold
        """only CEN used StandardScaler because the centroid of training set need
        to be move to origin"""
        #self.scaler = preprocessing.StandardScaler()
        self.metric = metric

    def fit(self, X):
        #self.scaler.fit(X)
        #X = self.scaler.transform(X)
        # because we are using StandardScaler, the centroid is a
        # vector of zeros, but we save it in shape (1, n) to allow
        # cdist to work happily later.
        self.centroid = np.zeros((1, X.shape[1]))
        # no need to scale again
        dists = self.get_density(X)
        # transform relative threshold (eg 95%) to absolute
        self.abs_threshold = np.percentile(dists, 100 * self.threshold)
    #It is actually the mean of the distances from each points in training set
    #to the centroid zero.
    def get_density(self, X):
        #if scale:
        #    X = self.scaler.transform(X)
        dists = scipy.spatial.distance.cdist(X, self.centroid, metric=self.metric)
        dists = np.mean(dists, axis=1)
        return dists

    def predict(self, X):
        dists = self.get_density(X)
        return dists > self.abs_threshold




"************************** NEGATIVE DISTANCE ********************************"
class NegativeMeanDistance:
    def __init__(self, metric="euclidean"):
        self.metric = metric

    def fit(self, X):
        self.X = X

    def score_samples(self, X):
        dists = scipy.spatial.distance.cdist(X, self.X, metric=self.metric)
        return -np.mean(dists, axis=1)


"*************************** DENSITY *****************************************"
class DensityBasedOneClassClassifier:
    def __init__(self, threshold=0.95,
                 kernel="gaussian",
                 bandwidth=1.0,
                 metric="euclidean",
                 should_downsample=False,
                 downsample_count=1000,
                 scale = "standard"):

        self.should_downsample = should_downsample
        self.downsample_count = downsample_count
        self.threshold = threshold

        #Load dataset, standard or maxabs[-1,1], minmax[0,1], No
        if (scale == "standard"):
            self.scaler = preprocessing.StandardScaler()
        elif (scale == "minmax"):
            self.scaler = preprocessing.MinMaxScaler()
        elif (scale == "maxabs"):
            self.scaler = preprocessing.MaxAbsScaler()

        if kernel == "really_linear":
            self.dens = NegativeMeanDistance(metric=metric)
        else:
            self.dens = KernelDensity(bandwidth=bandwidth, kernel=kernel, metric=metric)

    def fit(self, X):
        # scale
        self.scaler.fit(X)
        self.X = self.scaler.transform(X)

        # downsample?
        if self.should_downsample:
            self.X = self.downsample(self.X, self.downsample_count)

        self.dens.fit(self.X)
        # transform relative threshold (eg 95%) to absolute
        dens = self.get_density(self.X, scale=False) # no need to scale again
        self.abs_threshold = np.percentile(dens, 100 * (1 - self.threshold))

    def get_density(self, X, scale=True):
        if scale:
            X = self.scaler.transform(X)
        # in negative log-prob (for KDE), in negative distance (for NegativeMeanDistance)
        return self.dens.score_samples(X)

    def predict(self, X):
        dens = self.get_density(X)
        return dens < self.abs_threshold # in both KDE and NMD, lower values are more anomalous

    def downsample(self, X, n):
        # we've already fit()ted, but we're worried that our X is so
        # large our classifier will be too slow in practice. we can
        # downsample by running a kde on X and sampling from it (this
        # will be slow, but happens only once), and then using those
        # points as the new X.
        if len(X) < n:
            return X
        kde = KernelDensity()
        kde.fit(X)
        return kde.sample(n)

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Sun Aug 14 10:48:37 2016

@author: VANLOI
"""



def Plotting_AUC(name_dataset, path_result, training_size,
                 FPR_auto, TPR_auto, auc_auto,
                 FPR_cen, TPR_cen, auc_cen,
                 FPR_kde, TPR_kde, auc_kde):
    print ("\n*********************** Plot AUC *************************")
    plt.figure(figsize=(6,6))
    plt.title('The ROC curves - '+ name_dataset, fontsize=16)
    plt.plot(FPR_auto, TPR_auto, 'g-^'  , label='OCAE      (AUC = %0.3f)'% auc_auto, markevery = 150 , markersize = 6)
    plt.plot(FPR_cen, TPR_cen,   'b-o' ,  label='OCCEN    (AUC = %0.3f)'% auc_cen, markevery = 150 , markersize = 6)
    plt.plot(FPR_kde, TPR_kde, 'r-x' , label='OCKDE    (AUC = %0.3f)'% auc_kde, markevery = 150 , markersize = 6)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.1])
    plt.ylim([-0.1,1.1])
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.savefig(path_result + "fig_" + name_dataset +"_" + training_size + "_Auc.pdf")
    plt.show()

def Plotting_End2End_RE(RE, epoch, ymin, ymax, data_name, path):
    """Plotting RE on train_set and validation_set of the End-to-End traing
    process"""

    plt.figure(figsize=(6,3))
    #plt.title('End-to-End training RE on ' + data_name, fontsize=16)
    plt.xlim([0.0, epoch + 1.0])
    plt.ylim([ymin,ymax])

    x  = RE[:,0]
    y1 = RE[:,1]
    y2 = RE[:,2]
    plt.plot(x, y1,  'b', label = 'Validation set')
    plt.plot(x, y2,  'r', label = 'Training set')
    plt.legend(loc='upper right')
    plt.ylabel('Error', fontsize=14)
    plt.xlabel('Epochs', fontsize=14)
    plt.savefig(path + data_name +"_End2End_loss.pdf")
    plt.show()

def Plotting_Loss_Component(LOSS, RE, ymin, ymax, data_name, path):
    """Plotting RE on train_set and validation_set of the End-to-End traing
    process"""

    plt.figure(figsize=(6,3))
    #plt.title('End-to-End training RE on ' + data_name, fontsize=16)

    x  = LOSS[:,0]
    y1 = LOSS[:,1]
    y2 = LOSS[:,2]
    y3 = RE[:,2]

    plt.xlim([0.0, max(x) + 1.0])
    plt.ylim([ymin,ymax])

    plt.plot(x, y1,  'b', label = 'Recon error')
    plt.plot(x, y2,  'g', label = 'KL-divergence')
    plt.plot(x, y3,  'r', label = 'Training error')
    plt.legend(loc='upper right')
    plt.ylabel('Errors', fontsize=14)
    plt.xlabel('Epochs', fontsize=14)
    plt.savefig(path + data_name +"_traing_errors.pdf")
    plt.show()


"*****************************************************************************"
def Plotting_Monitor(RE, ymin, ymax, data_name, path):
    """Plotting RE on train_set and validation_set of the End-to-End traing
    process"""

    plt.figure(figsize=(6,3))
    #plt.title('Monitoring AUC every 5 epoch on ' + data_name, fontsize=16)
    ax = plt.subplot(111)

    x   = RE[:,0]
    lof = RE[:,1]
    cen = RE[:,2]
    dis = RE[:,3]
    kde = RE[:,4]
    svm5 = RE[:,5]
    svm1 = RE[:,6]
    ae  = RE[:,7]

    plt.xlim([0.0, max(x) + 1.0])
    plt.ylim([ymin,ymax])
    ax = plt.subplot(111)

    plt.plot(x, lof,  'r-o', label = 'LOF', markevery = 20 , markersize = 6)
    plt.plot(x, cen,  'b-x', label = 'CEN', markevery = 20 , markersize = 6)
    plt.plot(x, dis,  'g-^', label = 'MDIS', markevery = 20 , markersize = 6)
    plt.plot(x, kde,  'y-x', label = 'KDE', markevery = 20 , markersize = 6)
    plt.plot(x, svm5,  'r-^', label = 'SVM05', markevery = 20 , markersize = 6)
    plt.plot(x, svm1,  'g-o', label = 'SVM01', markevery = 20 , markersize = 6)
    plt.plot(x, ae,   'b-^', label = 'AE' , markevery = 20 , markersize = 6)

    ax.legend(bbox_to_anchor=(0.99, 0.28), ncol= 3, fontsize = 'medium')

    #plt.legend(loc='upper right')
    plt.ylabel('AUC', fontsize=14)
    plt.xlabel('Epochs', fontsize=14)
    plt.savefig(path + data_name + "_Monitor_AUCs.pdf")
    plt.show()


#def Plotting_Monitor(AUC_hidden, epoch, ymin, ymax, data_name, path):
#    """Plotting RE on train_set and validation_set of the End-to-End traing
#    process"""
#
#    plt.figure(figsize=(8,4))
#
#    x   = AUC_hidden[:,0]
#    lof = AUC_hidden[:,1]
#    cen = AUC_hidden[:,2]
#    dis = AUC_hidden[:,3]
#    kde = AUC_hidden[:,4]
#    svm5 = AUC_hidden[:,5]
#    svm1 = AUC_hidden[:,6]
#    ae  = AUC_hidden[:,7]
#
#    ax0 = plt.subplot2grid((1, 8), (0, 0), colspan=7)
#    plt.xlim([0.0, epoch + 1.0])
#    plt.ylim([ymin,ymax])
#    ax0.plot(x, lof,  'r-o', label = 'LOF', markevery = 20 , markersize = 6)
#    ax0.plot(x, cen,  'b-x', label = 'CEN', markevery = 20 , markersize = 6)
#    ax0.plot(x, dis,  'g-^', label = 'DIS', markevery = 20 , markersize = 6)
#    ax0.plot(x, kde,  'y-x', label = 'KDE', markevery = 20 , markersize = 6)
#    ax0.plot(x, svm5, 'r-^', label = 'SVM05', markevery = 20 , markersize = 6)
#    ax0.plot(x, svm1, 'g-o', label = 'SVM01', markevery = 20 , markersize = 6)
#    ax0.plot(x, ae,   'b-^', label = 'AE' , markevery = 20 , markersize = 6)
#
#    ax0.legend(bbox_to_anchor=(0.99, 0.32), ncol= 3)
#    #plt.legend(loc='upper right')
#    plt.ylabel('AUC', fontsize=14)
#    plt.xlabel('Epochs', fontsize=14)
#
#    ax1 = plt.subplot2grid((1, 8), (0, 7))
#    plt.ylim([ymin,ymax])
#    ax1.plot(x, lof,  'r-o', label = 'LOF', markevery = 20 , markersize = 6)
#    ax1.plot(x, cen,  'b-x', label = 'CEN', markevery = 20 , markersize = 6)
#    ax1.plot(x, dis,  'g-^', label = 'DIS', markevery = 20 , markersize = 6)
#    ax1.plot(x, kde,  'y-x', label = 'KDE', markevery = 20 , markersize = 6)
#    ax1.plot(x, svm5, 'r-^', label = 'SVM05', markevery = 20 , markersize = 6)
#    ax1.plot(x, svm1, 'g-o', label = 'SVM01', markevery = 20 , markersize = 6)
#    ax1.plot(x, ae,   'b-^', label = 'AE' , markevery = 20 , markersize = 6)
#    ax1.axes.get_yaxis().set_visible(False)
#    ax1.axes.get_xaxis().set_visible(False)
#
#    plt.tight_layout()
#    plt.savefig(path + data_name + "_Monitor_AUCs.pdf")
#    plt.show()

def Plotting_Pre_RE(RE, n_layers, epoch, ymin, ymax, batch_size, data_name):
    """Plotting REs of each dAE in the pre-training process"""
    plt.figure(figsize=(8,4))
    plt.title('Pre-training RE on' + data_name + '- Batch size = ' + str(batch_size), fontsize=16)
    plt.xlim([0.0, epoch + 1.0])
    plt.ylim([ymin,ymax])

    color = ['b', 'g', 'r', 'y']
    label = ["layer 1", "layer 2", "layer 3", "layer 4"]

    ax = plt.subplot(111)
    x  = RE[:,0]
    for i in range(n_layers):
        y = RE[:,i+1]
        plt.plot(x, y,  color[i], label = label[i])

    ax.legend(bbox_to_anchor=(0.99, 0.99), ncol=n_layers)
    #plt.legend(loc='upper right')
    plt.ylabel('Reconstruction errors', fontsize=14)
    plt.xlabel('Epochs', fontsize=14)
    plt.show()

"Plotting recontruction error from three autoencoders together"
def Plotting_Pre_RE1(re, stop_epoch, n_layers, ymin, ymax, batch_size, data_name, path):
    """Plotting REs of each dAE in the pre-training process"""
    plt.figure(figsize=(8,4))
    #plt.title('Pre-training RE on ' + data_name + ' - Batch size = ' + str(batch_size), fontsize=16)

    max_epoch = np.max(stop_epoch)

    plt.xlim([0.0, max_epoch + 1.0])
    plt.ylim([ymin,ymax])

    color = ['b', 'g', 'r', 'y']
    label = ["layer 1", "layer 2", "layer 3", "layer 4"]

    ax = plt.subplot(111)

    for i in range(n_layers):
        x = None
        y = None
        x = np.array(range(int(stop_epoch[i])))   #stop epoches of layer i
        y = re[:,i]                               #pre-train errors of layer i
        y = y[:len(x)]
        plt.plot(x, y,  color[i], label = label[i]) #plot pre-train errors of each layer

    ax.legend(bbox_to_anchor=(0.99, 0.99), ncol=n_layers)
    #plt.legend(loc='upper right')
    plt.ylabel('Reconstruction Error', fontsize=14)
    plt.xlabel('Epochs', fontsize=14)
    plt.savefig(path + data_name + "_Pre_train.pdf")
    plt.show()

#"Each subplot for ploting reconstruction error of each autoencoder"
#def Plotting_Pre_RE1(re, stop_epoch, n_layers, ymin, ymax, batch_size, data_name):
#    """Plotting REs of each dAE in the pre-training process"""
#
#    max_epoch = np.max(stop_epoch)
#    color = ['b-', 'g-', 'r-']
#    plt.subplots(ncols=3, nrows = 1, figsize=(8, 3))
#
#    for i in range(n_layers):
#        x = np.array(range(int(stop_epoch[i])))   #stop epoches of layer i
#        y = re[:,i]                               #pre-train errors of layer i
#        y = y[:len(x)]
#        fig = plt.subplot(1, 3, i+1)
#
#        plt.xlim([0.0, max_epoch + max_epoch/20])
#        plt.ylim([ymin,ymax])
#
#        plt.plot(x, y, color[i])
#        plt.legend(['Layer ' + str(i+1)])
#        if (i==0):
#            plt.ylabel('Reconstruction Error', fontsize=14)
#        else:
#            fig.axes.get_yaxis().set_visible(False)
#
#        plt.xlabel('Epochs', fontsize=14)
#
#        plt.yticks(fontsize=10)
#        plt.xticks(rotation = 30, fontsize=10)
#
#        #hide the first zero in axes matplotlib
#        ax = plt.gca()
#        xticks = ax.xaxis.get_major_ticks()
#        xticks[0].label1.set_visible(False)
#
#    plt.subplots_adjust(wspace=0.005, hspace=0)
#    plt.savefig(path + data_name + "_Pre_train.pdf")
#    plt.show()


def Plotting_AUC_RE(AUC_RE, dataset, ymin, ymax, path):
    """Plotting AUC against training-RE when evaluting the model. This is aim to
    do gridsearch over batch_sizes to choose the best performanced model.
    Hopfully, the smaller training-RE the model produces, the higher accuracy
    when evaluting the model on testing set"""
    plt.figure(figsize=(8,4))
    plt.title('AUC against RE - '+ dataset, fontsize=16)

    #Sorted AUC_RE by reconstruction error
    AUC_RE = AUC_RE[np.argsort(AUC_RE[:,9])]

    x = AUC_RE[:,9]
    plt.xlim( x[0] - (x[-1]-x[0])/20 , x[-1] + (x[-1]-x[0])/20)
    plt.ylim([ymin, ymax])

    y01 = AUC_RE[:,2]  #AUC of LOF
    y11 = AUC_RE[:,3]  #AUC of CEN
    y21 = AUC_RE[:,4]  #AUC of NDIS
    y31 = AUC_RE[:,5]  #AUC of KDE
    y41 = AUC_RE[:,6]  #AUC of KDE
    y51 = AUC_RE[:,8]  #AUC of AE


    ax = plt.subplot(111)

    plt.plot(x, y01,  'b-p', label = 'LOF', markersize = 6)
    plt.plot(x, y11,  'r-p', label = 'CEN', markersize = 6)
    plt.plot(x, y21,  'g-^', label = 'NDIS',markersize = 6)
    plt.plot(x, y31,  'y-d', label = 'KDE',markersize = 6)
    plt.plot(x, y41,  'r-s', label = 'SVM05',markersize = 6)
    plt.plot(x, y51,  'b-s', label = 'AE',markersize = 6)
    #b: blue | g: green | r: red | c: cyan | m: magenta | y: yellow | k: black | w: white

    ax.legend(bbox_to_anchor=(0.99, 0.25), ncol=3)
    plt.ylabel('AUC Value', fontsize=14)
    plt.xlabel('Reconstruction Error x 100', fontsize=14)
    plt.savefig(path + "AUC_RE_" + dataset +".pdf")
    plt.show()

def Plotting_AUC_Batch_Size(AUC_RE, dataset, ymin, ymax, path):
    """Plotting AUC against training-RE when evaluting the model. This is aim to
    do gridsearch over batch_sizes to choose the best performanced model.
    Hopfully, the smaller training-RE the model produces, the higher accuracy
    when evaluting the model on testing set"""
    plt.figure(figsize=(8,4))
    plt.title('AUC against RE - '+ dataset, fontsize=16)

    #Sorted AUC_RE by reconstruction error
    #AUC_RE = AUC_RE[np.argsort(AUC_RE[:,9])]

    x = AUC_RE[:,0]
    plt.xlim( x[0] - 1 , x[-1] + 1)
    plt.ylim([ymin, ymax])

    y01 = AUC_RE[:,2]  #AUC of LOF
    y11 = AUC_RE[:,3]  #AUC of CEN
    y21 = AUC_RE[:,4]  #AUC of NDIS
    y31 = AUC_RE[:,5]  #AUC of KDE
    y41 = AUC_RE[:,6]  #AUC of KDE
    y51 = AUC_RE[:,8]  #AUC of AE


    ax = plt.subplot(111)

    plt.plot(x, y01,  'b-p', label = 'LOF', markersize = 6)
    plt.plot(x, y11,  'r-p', label = 'CEN', markersize = 6)
    plt.plot(x, y21,  'g-^', label = 'NDIS',markersize = 6)
    plt.plot(x, y31,  'y-d', label = 'KDE',markersize = 6)
    plt.plot(x, y41,  'r-s', label = 'SVM05',markersize = 6)
    plt.plot(x, y51,  'b-s', label = 'AE',markersize = 6)
    #b: blue | g: green | r: red | c: cyan | m: magenta | y: yellow | k: black | w: white

    ax.legend(bbox_to_anchor=(0.99, 0.25), ncol=3)
    plt.ylabel('AUC Value', fontsize=14)
    plt.xlabel('Reconstruction Error x 100', fontsize=14)
    plt.savefig(path + "AUC_RE_" + dataset +".pdf")
    plt.show()

def Plotting_AUC_BW(AUC_Hidden, dataset, xmax, ymin, ymax, training_size, path ):
    plt.figure(figsize=(10,6))
    plt.title('AUC against BW - '+ dataset, fontsize=16)
    plt.xlim([0.0, xmax])
    plt.ylim([ymin, ymax])

    x   = AUC_Hidden[:,0]
    y11 = AUC_Hidden[:,1]
    y21 = AUC_Hidden[:,2]
    y31 = AUC_Hidden[:,3]
    y41 = AUC_Hidden[:,4]
    y51 = AUC_Hidden[:,5]

    plt.plot(x, y11,  'b-s', label = 'KDE      - Hidden',markersize = 6)
    plt.plot(x, y21,  'r-p', label = 'Negative Distance', markersize = 6)
    plt.plot(x, y31,  'g-^', label = 'SVM(0.5) - Hidden',markersize = 6)
    plt.plot(x, y41,  'y-d', label = 'SVM(0.2) - Hidden',markersize = 6)
    plt.plot(x, y51,  'm-s', label = 'SVM(0.1) - Hidden',markersize = 6)
    #b: blue | g: green | r: red | c: cyan | m: magenta | y: yellow | k: black | w: white
    plt.legend(loc='lower right')
    plt.ylabel('AUC Value', fontsize=14)
    plt.xlabel('Bandwidth', fontsize=14)
    plt.savefig(path + "AUC_BW_" + dataset + "_"+ training_size +".pdf")
    plt.show()


def plot_auc_size_input(data, data_name, sizes, path):

    # data to plot
    n_groups = 5
    LOF     = data[:,0:1]
    CEN     = data[:,1:2]
    DIS     = data[:,2:3]
    KDE     = data[:,3:4]
    SVM05   = data[:,4:5]
    SVM01   = data[:,-1]

    plt.figure(figsize=(6, 4))
    ax = plt.subplot(111)

#    plt.title(data_name + ' Attack Group', fontsize=16)
    plt.ylim([0.0, 1.0])
#    plt.ylim([0.0, m + m/5])

    index = np.arange(n_groups)
    bar_width = 0.1
    opacity = 1.0

    plt.bar(index + bar_width, LOF, bar_width,
                 alpha=opacity, color='b', label='LOF')

    plt.bar(index + 2*bar_width, CEN, bar_width,
                 alpha=opacity,color='g',label='CEN')

    plt.bar(index + 3*bar_width , DIS, bar_width,
                 alpha=opacity,color='r',label='DIS')

    plt.bar(index + 4*bar_width, KDE, bar_width,
                 alpha=opacity,color='y',label='KDE')

    plt.bar(index + 5*bar_width, SVM05, bar_width,
                 alpha=opacity,color='c',label='SVM05')

    plt.bar(index + 6*bar_width, SVM01, bar_width,
                 alpha=opacity,color='maroon',label='SVM01')

    ax.legend(bbox_to_anchor=(1.04, 0.42), ncol=1, fontsize = 'small')

    plt.xlabel('Size of training set', fontsize=16)
    plt.ylabel('AUC', fontsize=16)
    plt.yticks(fontsize=12)

    ax.yaxis.grid(True)

    plt.xticks(index + 2*bar_width, ('0.5%('+str(sizes[0])+')', '1%('+str(sizes[1])+')', '5%('+str(sizes[2])+')', '10%('+str(sizes[3])+')', '20%('+str(sizes[4])+')'),rotation=15,fontsize=12)

    plt.tight_layout()
    plt.savefig(path + data_name + "_auc_size.pdf")
    plt.show()




"Plot AUC vs Size of training data - 1"
def plot_auc_size_1(data, data_name, sizes, path):
    # data to plot
    n_groups = 5
    LOF     = data[:,0:1]
    CEN     = data[:,1:2]
    DIS     = data[:,2:3]
    KDE     = data[:,3:4]
    SVM05   = data[:,4:5]
    SVM01   = data[:,5:6]
#    RE      = data[:,-1]

    plt.figure(figsize=(6, 4))
    ax = plt.subplot(111)

#    plt.title(data_name + ' Attack Group', fontsize=16)
    plt.ylim([0.0, 1.0])
#    plt.ylim([0.0, m + m/5])

    index = np.arange(n_groups)
    bar_width = 0.1
    opacity = 1.0

    plt.bar(index + bar_width, LOF, bar_width,
                 alpha=opacity, color='cyan', label='LOF')

    plt.bar(index + 2*bar_width, CEN, bar_width,
                 alpha=opacity,color='yellow',label='CEN')

    plt.bar(index + 3*bar_width , DIS, bar_width,
                 alpha=opacity,color='magenta',label='NDIS')

    plt.bar(index + 4*bar_width, KDE, bar_width,
                 alpha=opacity,color='blue',label='KDE')

    plt.bar(index + 5*bar_width, SVM05, bar_width,
                 alpha=opacity,color='lightblue',label=r'SVM$_{\nu = 0.5}$')

    plt.bar(index + 6*bar_width, SVM01, bar_width,
                 alpha=opacity,color='plum',label=r'SVM$_{\nu = 0.1}$')

#    plt.bar(index + 7*bar_width, RE, bar_width,
#                 alpha=opacity,color='springgreen',label='RE-Based')
    #xx-small, x-small, small, medium, large, x-large, xx-large
    ax.legend(bbox_to_anchor=(1.0, 0.30), ncol=2, fontsize = 'large')

    plt.xlabel('Size of training set', fontsize=16)
    plt.ylabel('AUC', fontsize=16)
    plt.yticks(fontsize=12)

    ax.yaxis.grid(True)

    plt.xticks(index + 2*bar_width, (str(sizes[0]),\
                                     str(sizes[1]),\
                                     str(sizes[2]),\
                                     str(sizes[3]),\
                                     str(sizes[4])),\
                                     rotation=0,fontsize=12)

    plt.tight_layout()
    plt.savefig(path + data_name + "_auc_size.pdf")
    plt.show()

#from matplotlib import rcParams
##rcParams['mathtext.default'] = 'regular'
#rcParams['text.usetex']=True
# Mathtext font size
# \tiny, \small, \normalsize, \large, \Large, \LARGE, \huge and \Huge
"Plot AUC vs Size of training data - 2"
def plot_auc_size_2(data, data_name, name, sizes, method, path):
    cl = ["LOF", "CEN", "MDIS", "KDE", r'OCSVM$_{\nu=0.5}$', r'OCSVM$_{\nu=0.1}$']
    # data to plot
    n_groups = 6
    Z500    = data[:,0:1]
    Z1000     = data[:,1:2]
    Z2000     = data[:,2:3]
    Z5000     = data[:,3:4]
    Z10000      = data[:,-1]
#    RE      = data[:,-1]

    plt.figure(figsize=(6, 4))
    ax = plt.subplot(111)

    plt.title(name , fontsize=16)
    plt.ylim([0.5, 1.0])
#    plt.ylim([0.0, m + m/5])

    index = np.arange(n_groups)
    bar_width = 0.125
    space     = 0.0
    opacity = 1.0

    plt.bar(index + bar_width+space, Z500, bar_width,
                 alpha=opacity, color='cyan', label='500')

    plt.bar(index + 2*(bar_width+space), Z1000, bar_width,
                 alpha=opacity,color='yellow',label='1000')

    plt.bar(index + 3*(bar_width+space) , Z2000, bar_width,
                 alpha=opacity,color='magenta',label='2000')

    plt.bar(index + 4*(bar_width+space), Z5000, bar_width,
                 alpha=opacity,color='blue',label='5000')

    plt.bar(index + 5*(bar_width+space), Z10000, bar_width,
                 alpha=opacity,color='lightblue',label='10000')


#    plt.bar(index + 7*bar_width, RE, bar_width,
#                 alpha=opacity,color='springgreen',label='RE-Based')
    #xx-small, x-small, small, medium, large, x-large, xx-large
    ax.legend(bbox_to_anchor=(1.01, 0.61), ncol=1, fontsize = 'x-large')

#    plt.xlabel('Size of training set', fontsize=16)
    plt.ylabel('AUC', fontsize=16)
    plt.yticks(fontsize=14)

    ax.yaxis.grid(True)

    plt.xticks(index + 3*bar_width, (method + cl[0],\
                                     method + cl[1],\
                                     method + cl[2],\
                                     method + cl[3],\
                                     method + cl[4],\
                                     method + cl[5]),\
                                     rotation=20,fontsize=14)

    plt.tight_layout()
    plt.savefig(path + data_name + "_auc_size.pdf")
    plt.show()



"Visualize the hidden data in two dimension"
def visualize_hidden(train_set, test_set, actual, data_name, data, path):

    scaler = preprocessing.StandardScaler()
    scaler.fit(train_set)

    train_set = scaler.transform(train_set)
    test_set  = scaler.transform(test_set)

    test_X0 = test_set[(actual==1)]
    test_X1 = test_set[(actual==0)]

    plt.figure(figsize=(6, 6))
    ax = plt.subplot(111)
    if data == "train":
        plt.plot(train_set[:,0], train_set[:,1], 'bo', ms=5, mec="b", label="Normal Train")
    elif data == "normal":
        plt.plot(test_X0[:,0],   test_X0[:,1],   'go', ms=5, mec="g", label="Normal Test")
    else:
        plt.plot(test_X1[:,0],   test_X1[:,1],   'r^', ms=5, mec="r", label= "Anomaly Test")

    ax.legend(bbox_to_anchor=(1.0, 1.0), ncol=3 )

    plt.axis('equal')
    plt.ylim((-10.0, 10.0))
    plt.xlim((-10.0, 10.0))
    plt.tight_layout()
    #plt.savefig(path + data_name + "_v_hid_train_" + dataset + ".pdf")
    plt.show()
    plt.close



"Each subplot for ploting reconstruction error of each autoencoder"
def visualize_hidden1(train_set, test_set, actual, data_name, path):
    """Plotting REs of each dAE in the pre-training process"""

    scaler = preprocessing.StandardScaler()
    scaler.fit(train_set)

    train_set = scaler.transform(train_set)
    test_set  = scaler.transform(test_set)

    test_X0 = test_set[(actual==1)]
    test_X1 = test_set[(actual==0)]

    plt.subplots(ncols=3, nrows = 1, figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.axis('equal')
    plt.ylim((-10.0, 10.0))
    plt.xlim((-10.0, 10.0))
    plt.plot(train_set[:,0], train_set[:,1], 'bo', ms=5, mec="b", label="Normal Train")
    plt.legend(["Normal Train"])

    fig = plt.subplot(1, 3, 2)
    plt.axis('equal')
    plt.ylim((-10.0, 10.0))
    plt.xlim((-10.0, 10.0))
    plt.plot(test_X0[:,0],   test_X0[:,1],   'go', ms=5, mec="g", label="Normal Test")
    fig.axes.get_yaxis().set_visible(False)
    plt.legend(["Normal Test"])

    fig = plt.subplot(1, 3, 3)
    plt.axis('equal')
    plt.ylim((-10.0, 10.0))
    plt.xlim((-10.0, 10.0))
    plt.plot(test_X1[:,0],   test_X1[:,1],   'r^', ms=5, mec="r", label= "Anomaly Test")
    fig.axes.get_yaxis().set_visible(False)
    plt.legend(["Anomaly Test"])

    plt.subplots_adjust(wspace=0.05, hspace=0)
    plt.savefig(data_name + "_Visualize.pdf")
    plt.show()


"Investigate Bandwidth/gramma parameters of SVM and KDE"
def Plot_AUC_Bandwidth(auc, data_name, X_max, n_features ,path):

    bw    = auc[:,0]
    kde   = auc[:,1]
    svm05 = auc[:,2]
    svm01 = auc[:,3]
    default_bw = (n_features/2.0)**0.5

    fig = plt.figure(figsize=(8,4))
    ax1 = fig.add_subplot(111)

    plt.xlim([0.0, X_max])
    plt.ylim([0.40,1.0])

    plt.plot(bw, svm05,  'r-o', ms=6, mec="r", label =r'$\mathrm{SVM}_{\nu = 0.5}$', markevery = 3)
    plt.plot(bw, svm01,  'g-^', ms=6, mec="g", label =r'$\mathrm{SVM}_{\nu = 0.1}$', markevery = 3)
    plt.plot(bw, kde,    'b-x', ms=6, mec="b", label= 'KDE', markevery = 3)
    plt.legend(bbox_to_anchor=(1.01, 0.15), ncol=3)
    ax1.set_ylabel('AUC', fontsize=14)
    ax1.set_xlabel('Bandwidth', fontsize=14)


    ax2 = ax1.twiny()
    new_tick_locations = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
    def tick_function(bw):
        gamma = 1.0/(2.0*bw*bw)
        return ["%.3f" % z for z in gamma]

    ax2.set_xlim(ax1.get_xlim())
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(new_tick_locations))
    ax2.set_xlabel(r"Gamma($\gamma$) =  $1/(2*bandwidth^{2})$", fontsize=14 )


    sparse_data = ["Arrhythmia", "Spambase", "UNSW", "NSLKDD", "InternetAds"]
    if (data_name in sparse_data):
        x_text = 4.0
    else:
        x_text = 0.5
    ax1.annotate('default value',
            xy=(default_bw, 1.0), xytext=(x_text, 1.06),
            arrowprops=dict(facecolor='green', arrowstyle="->"))

    plt.tight_layout()
    plt.savefig(path + "Bandwith_auc/" + data_name + "_BW.pdf")
    plt.show()



def plot_sparsity_auc_bar(data, improve_auc, spa_score, method, path):

    #Using CTU13-13 to demonstrate for four CTU13 datasets, [6,7,8,9]
    id_data = [0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13]
    improve_auc = improve_auc[id_data]
    spa_score   = spa_score[id_data]        #sparsity score
    labels      = data[id_data]

    n_groups = len(id_data)
    LOF     = improve_auc[:,2:3]
    CEN     = improve_auc[:,3:4]
    DIS     = improve_auc[:,4:5]
    KDE     = improve_auc[:,5:6]
    SVM05   = improve_auc[:,6:7]
    SVM01   = improve_auc[:,7:8]

    plt.figure(figsize=(8, 4))
    ax = plt.subplot(111)

    #plt.title(data_name + ' Attack Group', fontsize=16)
    plt.ylim([-0.45, 0.45])

    index = np.arange(n_groups)
    bar_width = 0.1
    opacity = 1.0

    plt.bar(index + 1*bar_width,   LOF,   bar_width, alpha=opacity, color='b', label='LOF')
    plt.bar(index + 2*bar_width, CEN,   bar_width, alpha=opacity, color='g',label='CEN')
    plt.bar(index + 3*bar_width, DIS,   bar_width, alpha=opacity, color='r',label='NDIS')
    plt.bar(index + 4*bar_width, KDE,   bar_width, alpha=opacity, color='y',label='KDE')
    plt.bar(index + 5*bar_width, SVM05, bar_width, alpha=opacity, color='c',label='SVM05')
    plt.bar(index + 6*bar_width, SVM01, bar_width, alpha=opacity, color='maroon',label='SVM01')
    #xx-small, x-small, small, medium, large, x-large, xx-large
    ax.legend(bbox_to_anchor=(0.44, 1.0), ncol=3, fontsize = 'medium')

    plt.xlabel('Sparsity of data', fontsize=14)
    plt.ylabel('($\mathrm{AUC}_{\mathrm{hidden}}$' + '-' + '$\mathrm{AUC}_{\mathrm{input}}$)', fontsize=14)
    plt.yticks(fontsize=12)

    ax.yaxis.grid(True)
    plt.xticks(index + 3*bar_width,(str(spa_score[i][1]) + '-' + str(labels[i]) for i in range(n_groups)),rotation=60,fontsize=11)

    plt.tight_layout()
    plt.savefig(path + "auc_sparsity_" + method + "_bar.pdf")
    plt.show()


def plot_sparsity_auc(data, improve_auc, spa_score, method, path):

    #Using CTU13-13 to demonstrate for four CTU13 datasets, [6,7,8,9]
    id_data = [0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 13]
    improve_auc = improve_auc[id_data]
    spa_score   = spa_score[id_data]        #sparsity score
    labels = data[id_data]                  #name of datasets

    LOF     = improve_auc[:,2:3]
    CEN     = improve_auc[:,3:4]
    DIS     = improve_auc[:,4:5]
    KDE     = improve_auc[:,5:6]
    SVM05   = improve_auc[:,6:7]
    SVM01   = improve_auc[:,7:8]

    plt.figure(figsize=(8, 4.5))
    ax = plt.subplot(111)


    plt.ylim([-0.4, 0.4])
    plt.xlim([-0.02, max(spa_score[:,1])+0.01])
    plt.xticks(spa_score[:,1],spa_score[:,1], rotation=90)

    #'b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'
    plt.plot(spa_score[:,1], LOF,   'b-s', ms=4, mec="b", label ='LOF', markevery = 1)
    plt.plot(spa_score[:,1], CEN,   'r-p', ms=4, mec="r", label ='CEN', markevery = 1)
    plt.plot(spa_score[:,1], DIS,   'g-^', ms=4, mec="g", label= 'NDIS', markevery = 1)
    plt.plot(spa_score[:,1], KDE,   'y-d', ms=4, mec="y", label ='KDE', markevery = 1)
    plt.plot(spa_score[:,1], SVM05, 'm-o', ms=4, mec="m", label =r'$\mathrm{SVM}_{\nu = 0.5}$', markevery = 1)
    plt.plot(spa_score[:,1], SVM01, 'c-x', ms=4, mec="c", label= r'$\mathrm{SVM}_{\nu = 0.1}$', markevery = 1)

    #xx-small, x-small, small, medium, large, x-large, xx-large
    ax.legend(bbox_to_anchor=(0.47, 1.0), ncol=3, fontsize = 'medium')

    plt.xlabel('Sparsity of data', fontsize=14)
    plt.ylabel('($\mathrm{AUC}_{\mathrm{hidden}}$' + '-' + '$\mathrm{AUC}_{\mathrm{input}}$)', fontsize=14)
    plt.yticks(fontsize=12)
    ax.yaxis.grid(True)

    ax.twiny()
    plt.xlim([-0.02, max(spa_score[:,1])+0.01])
    plt.xticks(spa_score[:,1], labels, rotation=90)

    plt.tight_layout()
    plt.savefig(path + "auc_sparsity_" + method + "_line.pdf")
    plt.show()



def plot_dimension_auc(data, improve_auc, spa_dim, method, path):

    #Using CTU13-13 to demonstrate for four CTU13 datasets, [6,7,8,9]
    id_data = [0, 1, 2, 4, 5, 9, 10, 11, 12, 13]
    improve_auc = improve_auc[id_data]
    spa_dim     = spa_dim[id_data]        #sparsity score

    improve_auc = np.insert(improve_auc, [0], spa_dim, axis=1)

    #improve_auc = sorted(improve_auc, key=lambda a_entry: a_entry[2])
    improve_auc = improve_auc[improve_auc[:,2].argsort()]
    dim     = np.asanyarray(improve_auc[:,2], dtype = int)
    idx     = np.asanyarray(improve_auc[:,0], dtype = int)
    labels1 = []
    for d in idx:
      labels1 = np.append(labels1, data[d])

                     #name of datasets

    LOF     = improve_auc[:,5:6]
    CEN     = improve_auc[:,6:7]
    DIS     = improve_auc[:,7:8]
    KDE     = improve_auc[:,8:9]
    SVM05   = improve_auc[:,9:10]
    SVM01   = improve_auc[:,10:11]

    plt.figure(figsize=(8, 4))
    ax = plt.subplot(111)

    log_dim = np.round(np.log(dim+1-9), 2)

    plt.ylim([-0.45, 0.45])
    plt.xlim([-0.1, max(log_dim)+0.1])
    plt.xticks(log_dim, dim, rotation=90)

    #'b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'
    plt.plot(log_dim, LOF,   'b-s', ms=4, mec="b", label ='LOF', markevery = 1)
    plt.plot(log_dim, CEN,   'r-p', ms=4, mec="r", label ='CEN', markevery = 1)
    plt.plot(log_dim, DIS,   'g-^', ms=4, mec="g", label= 'NDIS', markevery = 1)
    plt.plot(log_dim, KDE,   'y-d', ms=4, mec="y", label ='KDE', markevery = 1)
    plt.plot(log_dim, SVM05, 'm-o', ms=4, mec="m", label =r'$\mathrm{SVM}_{\nu = 0.5}$', markevery = 1)
    plt.plot(log_dim, SVM01, 'c-x', ms=4, mec="c", label= r'$\mathrm{SVM}_{\nu = 0.1}$', markevery = 1)

    #xx-small, x-small, small, medium, large, x-large, xx-large
    ax.legend(bbox_to_anchor=(0.47, 1.0), ncol=3, fontsize = 'medium')

    plt.xlabel('Dimension in log scale', fontsize=14)
    plt.ylabel('($\mathrm{AUC}_{\mathrm{hidden}}$' + '-' + '$\mathrm{AUC}_{\mathrm{input}}$)', fontsize=14)
    plt.yticks(fontsize=12)
    ax.yaxis.grid(True)

    ax.twiny()
    plt.xlim([-0.1, max(log_dim)+0.1])
    plt.xticks(log_dim, labels1, rotation=90)

    plt.tight_layout()
    plt.savefig(path + "auc_dimension_" + method + ".pdf")
    plt.show()



"****************** Plot histogram of z_mu, z_var and z **********************"
def histogram_z(x, name, alpha, epoch, path):

    mu    = np.mean(x)
    sigma = np.std(x)

    # the histogram of the data
    n, bins, patches = plt.hist(x, 20, normed=1, facecolor='green', alpha=0.5)
    # add a 'best fit' line
    y = mlab.normpdf( bins, mu, sigma)
    plt.plot(bins, y, 'r--', linewidth=1)

    if (name == 'mu'):
        title = r'$\mathrm{Histogram\ of\ \mu}_{\mathrm{z}}\ (\mathrm{\alpha\ = ' + str(alpha) + ',}\ \mathrm{epoch\ = }'+ str(epoch) + ')$'
        xlabel = r'$\mathrm{\mu}_{\mathrm{z}}$'
    elif (name == 'var'):
        title = r'$\mathrm{Histogram\ of\ \sigma}_{\mathrm{z}}\ (\mathrm{\alpha\ = ' + str(alpha) + ',}\ \mathrm{epoch\ = }'+ str(epoch) + ')$'
        xlabel = r'$\mathrm{\sigma}_{\mathrm{z}}$'
    else:
        title = r'$\mathrm{Histogram\ of\ z}\ (\mathrm{\alpha\ = ' + str(alpha) + ',}\ \mathrm{epoch\ = }'+ str(epoch) + ')$'
        xlabel = r'$\mathrm{z}$'

    plt.xlabel(xlabel, fontsize=18)
    plt.ylabel('Probability', fontsize=14)

    plt.title(title, fontsize=18)
    plt.axis([-3, 3, 0, max(y)+ 0.1*max(y)])
    plt.grid(True)
    plt.savefig(path + "Visualize_histogram/" + "his_" + name + "_" + str(alpha) + "_" +  str(epoch) + ".pdf")
    plt.show()

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Mon Nov 28 09:41:21 2016

@author: VANLOI
"""



def auc_density(training_set, testing_set, actual, scale):
    """Compute AUC for density-based methods: Centroid, Negative Mean Distances,
    Kernel Density Estimation and One-class Support Vector Machine, and LOF.
    """
    #gamma = 1/2bw^2 = 1/n_feautes -> bw = (n_features/2)^0.5
    #h_default = (d/2.0)**0.5
    bw = (training_set.shape[1]/2.0)**0.5        #default value in One-class SVM
    gamma = 1/(2*bw*bw)

    "*************** Centroid AE - Hidden layer **************"
    CEN = CentroidBasedOneClassClassifier()
    CEN.fit(training_set)
    predictions_cen = -CEN.get_density(testing_set)
    FPR_cen, TPR_cen, thresholds_cen = roc_curve(actual, predictions_cen)
    cen = auc(FPR_cen, TPR_cen)


    "****************** Negative Distance - Hidden layer **********************"
    clf_dis = DensityBasedOneClassClassifier(bandwidth = bw,
                                             kernel="really_linear",
                                             metric="euclidean",
                                             scale = scale)
    clf_dis.fit(training_set)
    predictions_dis  = clf_dis.get_density(testing_set)
    FPR_dis, TPR_dis, thresholds_dis = roc_curve(actual, predictions_dis)
    dis = auc(FPR_dis, TPR_dis)


    "****************** KDE AE - Hidden layer*****************"
    #  ['gaussian'|'tophat'|'epanechnikov'|'exponential'|'linear'|'cosine']
    KDE = DensityBasedOneClassClassifier(bandwidth = bw,
                                         kernel="gaussian",
                                         metric="euclidean",
                                         scale = scale)
    KDE.fit(training_set)
    predictions_kde = KDE.get_density(testing_set)
    FPR_kde, TPR_kde, thresholds_kde = roc_curve(actual, predictions_kde)
    kde = auc(FPR_kde, TPR_kde)


    "********************* 1-SVM Hidden layer ***************************"
    training_set, testing_set =  normalize_data(training_set, testing_set, scale)

    clf_05 = svm.OneClassSVM(nu=0.5, kernel="rbf", gamma=gamma)
    clf_05.fit(training_set)
    #n_support_vectors =  len(clf.support_vectors_)
    predictions_svm  = clf_05.decision_function(testing_set)
    FPR_svm, TPR_svm, thresholds_svm = roc_curve(actual, predictions_svm)
    svm_05 = auc(FPR_svm, TPR_svm)

    "nu = 0.1"
    clf_01 = svm.OneClassSVM( nu=0.1, kernel="rbf", gamma=gamma)
    clf_01.fit(training_set)
    #num_01 =  len(clf_01.support_vectors_)
    predictions_svm_01  = clf_01.decision_function(testing_set)
    FPR_svm_01, TPR_svm_01, thresholds_svm_01 = roc_curve(actual, predictions_svm_01)
    svm_01 = auc(FPR_svm_01, TPR_svm_01)

    "******************************* LOF **********************************"
    neighbors = (int)(len(training_set)*0.1)
    clf_lof = LocalOutlierFactor(n_neighbors=neighbors)
    clf_lof.fit(training_set)
    predict = clf_lof._decision_function(testing_set)
    FPR, TPR, thresholds = roc_curve(actual, predict)
    lof = auc(FPR, TPR)

    return lof, cen, dis, kde, svm_05, svm_01


def auc_AEbased(test_X, output_test, actual):

    "******************* Testing Output layer ***************"
    OF = -(((test_X - output_test)**2).mean(1))
    """Classification decision will be based on the error (MAE or RMSE) between
    output and input. The higher error value a example has, the stronger decision
    the example belongs to anomaly class.
    Because we set normal class is positive class, so we put minus "-" to MSE to
    make OF of normal examples are large while those from anomaly examples are small
    """
    predictions_auto = OF
    FPR_auto, TPR_auto, thresholds_auto = roc_curve(actual, predictions_auto)
    auc_auto = auc(FPR_auto, TPR_auto)

    return auc_auto

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Tue Aug 15 19:05:51 2017

@author: VANLOI
"""


# from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

path = "Results/"

"Check whether weights matrix is updated or not"


def check_weight_update(sda):
    np.set_printoptions(precision=4, suppress=True)
    "Check whether weights matrix is updated or not"
    for i in range(sda.n_layers):
        print("\n %d" % i)
        print(sda.Autoencoder_layers[i].W.get_value(borrow=True))

    for j in range(sda.n_layers, 2 * sda.n_layers):
        print("\n %d" % j)
        print(sda.Autoencoder_layers[j].W.eval())

    print("\n ************************************ ")


class SdA(object):

    def __init__(self, numpy_rng, theano_rng=None, n_ins=100, hidden_layers_sizes=[50, 30]):

        self.encoder = []
        self.decoder = []
        self.params = []
        self.n_layers = len(hidden_layers_sizes)

        """set seed one time per dataset, rng will randomly generate different
        number in each mini_batch_size and, different in each epoch. This is
        repetive the same number when we create new SdA object. See theano_rondom.py"""
        self.rng = theano.tensor.shared_randomstreams.RandomStreams(numpy_rng.randint(2 ** 30))

        assert self.n_layers > 0
        #        if not theano_rng:
        #            theano_rng = RandomStreams(numpy_rng.randint(2 ** 30))

        # allocate symbolic variables for the data
        # the data is presented as rasterized images
        self.x = T.matrix('x')

        for i in range(self.n_layers):
            # the size of the input is either the number of hidden units of
            # the layer below or the input size if we are on the first layer
            if (i == 0):
                input_size = n_ins
                layer_input = self.x
            else:
                input_size = hidden_layers_sizes[i - 1]
                layer_input = self.encoder[-1].output

            "Not the middle hidden layer"
            if (i < self.n_layers - 1):
                encoder_layer = HiddenLayer(rng=numpy_rng,
                                            input=layer_input,
                                            n_in=input_size,
                                            n_out=hidden_layers_sizes[i],
                                            activation=T.tanh)
                self.encoder.append(encoder_layer)
                self.params.extend(encoder_layer.params)

            else:
                "The middle hidden layer, linear"
                encoder_mu = HiddenLayer(rng=numpy_rng,
                                         input=self.encoder[i - 1].output,
                                         n_in=input_size,
                                         n_out=hidden_layers_sizes[i])

                encoder_var = HiddenLayer(rng=numpy_rng,
                                          input=self.encoder[i - 1].output,
                                          n_in=input_size,
                                          n_out=hidden_layers_sizes[i])

                self.encoder.append(encoder_mu)
                self.params.extend(encoder_mu.params)

                self.encoder.append(encoder_var)
                self.params.extend(encoder_var.params)

        "*************** Sample z **************"
        mu = self.encoder[-2].output
        log_var = self.encoder[-1].output
        sample_z = self.sample_z(mu, log_var)

        "*************** Decoder ***************"
        i = self.n_layers - 1
        while (i >= 0):
            input_size = hidden_layers_sizes[i]
            if (i > 0):
                output_size = hidden_layers_sizes[i - 1]
            else:
                output_size = n_ins

            if (i == self.n_layers - 1):  # the first layer in decoder
                layer_input = sample_z
                decoder_layer = HiddenLayer(rng=numpy_rng,
                                            input=layer_input,
                                            n_in=input_size,
                                            n_out=output_size,
                                            activation=T.tanh)  # may be linear
                self.decoder.append(decoder_layer)
                self.params.extend(decoder_layer.params)
            else:
                layer_input = self.decoder[-1].output
                decoder_layer = HiddenLayer(rng=numpy_rng,
                                            input=layer_input,
                                            n_in=input_size,
                                            n_out=output_size,
                                            activation=T.tanh,
                                            W=self.encoder[i].W.T)

                self.decoder.append(decoder_layer)
                self.params.append(decoder_layer.b)
            i = i - 1

        "******************* End To End Cost function ************************"
        z_mu = self.encoder[-2].output
        z_var = self.encoder[-1].output
        y = self.decoder[-1].output

        self.alpha = 1e-8
        self.lamda = 0.05

        self.recon = (((self.x - y) ** 2).mean(1)).mean()
        """When compute a constant together with theano variable, it will be converted
        into the same shape as the theano variable. We may compute mean over features
        of each example instead of sum. This is to avoid the difference in dimension of
        each data. Default lamda = 0.05, alpha = 1e-8"""

        alpha = self.alpha
        self.KL = T.mean(
            (0.5 / alpha) * T.mean(T.exp(z_var) + z_mu ** 2 - alpha - alpha * z_var + alpha * T.log(alpha), 1))
        self.end2end_cost = self.recon + self.lamda * T.log10(self.KL + 1)

        # Experiment: lamda = 0.05; alpha = 1e-8    1
        # mean(1) is within example, mean(0) is within each feature

    "**************************** Sample z ***********************************"

    def sample_z(self, mu, log_var):
        eps = self.rng.normal(mu.shape, 0.0, 1.0, dtype=theano.config.floatX)
        sample_z = mu + T.exp(log_var / 2) * eps
        return sample_z

    "********************** Compute KL and Recon Loss *************************"

    #    def Recon_KL_Loss(self, data_set):
    #        data_size = data_set.get_value().shape[0]
    #        index = T.lscalar('index')
    #        KL1 = self.lamda*T.log10(self.KL+1)
    #        Loss = theano.function([index],
    #                               outputs = [self.recon, KL1],
    #                               givens={self.x: data_set[index : data_size]})
    #        return Loss(0)

    def Recon_KL_loss_batch(self, train_x, batch_size):

        index = T.lscalar('index')
        # begining of a batch, given `index`
        batch_begin = index * batch_size
        # ending of a batch given `index`
        batch_end = batch_begin + batch_size
        KL1 = self.lamda * T.log10(self.KL + 1)
        loss_com = theano.function([index],
                                   outputs=[self.recon, KL1],
                                   givens={self.x: train_x[batch_begin: batch_end]})
        return loss_com

    def Recon_KL_Loss(self, train_x, batch_size):
        n_train = train_x.get_value().shape[0]
        n_batches = (int)(n_train / batch_size)
        loss_com = self.Recon_KL_loss_batch(train_x, batch_size)
        loss = np.empty([0, 2])
        for batch_index in range(n_batches):
            l = loss_com(index=batch_index)
            loss = np.append(loss, [l[0], l[1]])
        loss = np.reshape(loss, (-1, 2))

        return (loss.mean(0))

    "************************** Get Mu and Log_var ****************************"

    def get_mu_logvar(self, data_set):
        data_size = data_set.get_value().shape[0]
        index = T.lscalar('index')
        mu = self.encoder[-2].output
        log_var = self.encoder[-1].output
        mu_logvar = theano.function([index],
                                    outputs=[mu, log_var],
                                    givens={self.x: data_set[index: data_size]})
        return mu_logvar(0)

    "****** Error on train_x and valid_x before optimization process **********"

    def Loss_train_valid(self, train_x, valid_x):
        index = T.lscalar('index')

        train_size = train_x.get_value().shape[0]
        tm = theano.function([index],
                             outputs=self.end2end_cost,
                             givens={self.x: train_x[index: train_size]})

        valid_size = valid_x.get_value().shape[0]
        vm = theano.function([index],
                             outputs=self.end2end_cost,
                             givens={self.x: valid_x[index: valid_size]})

        return tm(0), vm(0)

    "**************************** Get hidden data z **************************"

    def get_hidden_data(self, data_set):
        data_size = data_set.get_value().shape[0]
        index = T.lscalar('index')
        mu = self.encoder[-2].output
        log_var = self.encoder[-1].output
        z = self.sample_z(mu, log_var)
        hidden_data = theano.function([index],
                                      outputs=z,
                                      givens={self.x: data_set[index: data_size]})
        return hidden_data(0)

    "************get data from the output of Autoencoder**********************"

    def get_output_data(self, data_set):
        data_size = data_set.get_value().shape[0]
        index = T.lscalar('index')
        y_data = theano.function([index],
                                 outputs=self.decoder[-1].output,
                                 givens={self.x: data_set[index: data_size]})
        return y_data(0)

    "******************** Histogram z, z_mu and z_var ************************"

    def Plot_histogram_z(self, train_set, test_set, actual, epoch, path):
        z_train = self.get_hidden_data(train_set)
        mu, logvar = self.get_mu_logvar(train_set)

        np.savetxt(path + "Visualize_histogram/" + "z_train_" + str(epoch) + "_" + str(self.alpha) + ".csv", z_train,
                   delimiter=",", fmt='%f')
        #        np.savetxt(path + "Visualize_histogram/" + "z_mu" + str(epoch) + ".csv",  mu, delimiter=",", fmt='%f' )
        #        np.savetxt(path + "Visualize_histogram/" + "z_var" + str(epoch) + ".csv", np.exp(logvar), delimiter=",", fmt='%f' )

        #        histogram_z(mu[:,0],             'mu' , self.alpha, epoch, path)
        #        histogram_z(np.exp(logvar[:,0]), 'var', self.alpha, epoch, path)
        histogram_z(z_train[:, 0], 'z', self.alpha, epoch, path)

    "********************** Standard deviation of z ***************************"

    def Compute_Std(self, train_set, test_set, actual, data_name, path):
        z_train = self.get_hidden_data(train_set)
        z_test = self.get_hidden_data(test_set)

        visualize_hidden1(z_train, z_test, actual, data_name, path)
        # std on each feature over data
        std = np.std(z_train, axis=0)
        np.set_printoptions(precision=6, suppress=True)
        print("\n+ Standard Deviation of Hidden data:")
        print(std)

    "********************* Compute AUC on hidden data *************************"

    def Compute_AUC_Hidden(self, train_set, test_set, actual, norm, data_name):
        z_train = self.get_hidden_data(train_set)  # get hidden values
        z_test = self.get_hidden_data(test_set)  # get hidden values
        y_test = self.get_output_data(test_set)  # get prediction values
        "Compute performance of classifiers on latent data"
        lof, cen, dis, kde, svm05, svm01 = auc_density(z_train, z_test, actual, norm)
        ae = auc_AEbased(test_set.get_value(), y_test, actual)
        return lof, cen, dis, kde, svm05, svm01, ae

    "**************************************************************************"

    def Save_Hidden_Data(self, train_set, test_set, data_name, path):
        z_train = self.get_hidden_data(train_set)  # get hidden values
        z_test = self.get_hidden_data(test_set)  # get hidden values
        np.savetxt(path + data_name + "_train_z.csv", z_train, delimiter=",", fmt='%f')
        np.savetxt(path + data_name + "_test_z.csv", z_test, delimiter=",", fmt='%f')

    "**************************************************************************"

    def Save_Hidden_Data_Size(self, train_set, test_set, data_name, size, path):
        z_train = self.get_hidden_data(train_set)  # get hidden values
        z_test = self.get_hidden_data(test_set)  # get hidden values
        np.savetxt(path + "data/" + data_name + "_train_z_" + str(size) + ".csv", z_train, delimiter=",", fmt='%f')
        np.savetxt(path + "data/" + data_name + "_test_z_" + str(size) + ".csv", z_test, delimiter=",", fmt='%f')

    "******** Training End-to-End Early-stopping by Downhill Package *********"

    def End2end_Early_stopping(self, numpy_rng, dataset, n_validate, data_name,
                               batch_size, end2end_lr, algo, norm, patience, validation):

        train_X, test_X, actual = dataset
        valid_x = train_X.get_value()[:n_validate]
        train_x = train_X.get_value()[n_validate:]
        "for compute tm and vm before optimization process"
        t = theano.shared(numpy.asarray(train_x, dtype=theano.config.floatX), borrow=True)
        v = theano.shared(numpy.asarray(valid_x, dtype=theano.config.floatX), borrow=True)

        "Training network by downhill"
        # 'adadelta' 'adagrad (default 0.01)' 'adam''esgd' 'nag''rmsprop' 'rprop' 'sgd'
        opt = downhill.build(algo=algo, params=self.params, loss=self.end2end_cost, inputs=[self.x])
        train = downhill.Dataset(train_x, batch_size=batch_size, rng=numpy_rng)
        valid = downhill.Dataset(valid_x, batch_size=len(valid_x), rng=numpy_rng)

        "***** Monitor before optimization *****"
        stop_ep = 0
        RE = np.empty([0, 3])
        monitor = np.empty([0, 8])
        #        LOSS = np.empty([0,3])
        #        self.Plot_histogram_z(train_X, test_X, actual, 0, path)
        # AUC before optimization
        lof, cen, dis, kde, svm05, svm01, ae = self.Compute_AUC_Hidden(train_X, test_X, actual, norm, data_name)
        a = np.column_stack([0, lof, cen, dis, kde, svm05, svm01, ae])
        monitor = np.append(monitor, a)
        # Loss components before optimization

        #        loss = self.Recon_KL_Loss(t, batch_size)
        #        LOSS = np.append(LOSS,[stop_ep, loss[0], loss[1]])
        # Error before optimization
        tm1, vm1 = self.Loss_train_valid(t, v)
        RE = np.append(RE, np.column_stack([stop_ep, vm1, tm1]))

        for tm1, vm1 in opt.iterate(train,  # 5, 5, 1e-2, 0.9
                                    valid,
                                    patience=patience,  # 10
                                    validate_every=validation,  # 5
                                    min_improvement=1e-3,  # 1e-3
                                    # learning_rate =  end2end_lr,  # 1e-4
                                    momentum=0.0,
                                    nesterov=False):
            stop_ep = stop_ep + 1
            #            loss = self.Recon_KL_Loss(t, batch_size)
            #            LOSS = np.append(LOSS,[stop_ep, loss[0], loss[1]])
            #            "******* Monitor optimization ******"
            if ((stop_ep % 200 == 0) and (stop_ep > 0)):
                # self.Plot_histogram_z(train_X, test_X, actual, stop_ep, path)
                lof, cen, dis, kde, svm05, svm01, ae = self.Compute_AUC_Hidden(train_X, test_X, actual, norm, data_name)
                a = np.column_stack([stop_ep, lof, cen, dis, kde, svm05, svm01, ae])

            monitor = np.append(monitor, a)
            re = np.column_stack([stop_ep, vm1['loss'], tm1['loss']])
            RE = np.append(RE, re)

            if (stop_ep >= 1000):
                break

        # Plotting AUC and save to csv file
        monitor = np.reshape(monitor, (-1, 8))
        #        Plotting_Monitor(monitor, 0.4, 1.0, data_name, path)
        #        np.savetxt(path + data_name + "_monitor_auc1.csv", monitor, delimiter=",", fmt='%f' )

        #        LOSS = np.reshape(LOSS, (-1,3))
        #        Plotting_Loss_Component(LOSS, RE, 0.0, 0.5, data_name, path)
        #        np.savetxt(path + data_name + "_loss_component.csv", LOSS, delimiter=",", fmt='%f' )

        RE = np.reshape(RE, (-1, 3))
        #        Plotting_End2End_RE(RE, stop_ep, 0.0, 0.4, data_name, path)
        #        np.savetxt(path +  data_name + "_training_error1.csv", RE, delimiter=",", fmt='%f' )

        np.set_printoptions(precision=6, suppress=True)
        print("\n ", RE[stop_ep])

        return RE[stop_ep]


def test_SdA(pre_lr=0.01, end2end_lr=1e-4, algo='sgd',
             dataset=[], data_name="WBC", n_validate=0, norm="maxabs",
             batch_size=10, hidden_sizes=[1, 1, 1], corruptions=[0.0, 0.0, 0.0],
             patience=1, validation=1):
    numpy_rng = numpy.random.RandomState(89677)  # numpy random generator 89677
    train_X, test_X, actual = dataset  # dataset is already normalised

    input_size = train_X.get_value().shape[1]  # input size = dimension
    train_x = train_X.get_value()[n_validate:]  # 80% for pre-training, 20% for validation
    n_train_batches = train_x.shape[0]
    n_train_batches //= batch_size  # number of batches for pre-training

    # construct the stacked denoising autoencoder class
    sda = SdA(numpy_rng=numpy_rng, n_ins=input_size,
              hidden_layers_sizes=hidden_sizes)

    RE = sda.End2end_Early_stopping(numpy_rng, dataset, n_validate, data_name,
                                    batch_size, end2end_lr, algo, norm, patience, validation)
    return sda, RE


def Main_Test():
    '''    list_data = ["PageBlocks", "WPBC", "PenDigits", "GLASS", "Shuttle", "Arrhythmia", \
                     "CTU13_10", "CTU13_08", "CTU13_09", "CTU13_13", \
                     "Spambase", "UNSW", "NSLKDD", "InternetAds"]'''
    list_data=["UNSW"]
    norm = "maxabs"  # standard, maxabs[-1,1] or minmax[0,1]
    corruptions = [0.1, 0.1, 0.1]

    print("+ VAE: 0.05, Group")
    print("+ Data: ", list_data)
    print("+ Scaler: ", norm)

    AUC_Hidden = np.empty([0, 10])  # store auc of all hidden data
    num = 0  # a counter
    for data in list_data:
        num = num + 1

        h_sizes = hyper_parameters(data)  # Load hyper-parameters
        train_set, test_set, actual = load_data(data)  # load original data

        train_X, test_X = normalize_data(train_set, test_set, norm)  # Normalize data

        train_X = theano.shared(numpy.asarray(train_X, dtype=theano.config.floatX), borrow=True)
        test_X = theano.shared(numpy.asarray(test_X, dtype=theano.config.floatX), borrow=True)

        datasets = [(train_X), (test_X), (actual)]  # Pack data for training AE

        in_dim = train_set.shape[1]  # dimension of input data
        n_vali = (int)(train_set.shape[0] / 5)  # size of validation set
        n_train = len(train_set) - n_vali  # size of training set
        # batch    = int(n_train/20)                           #Training set will be split training set into 20 batches
        # print data information
        pat, val, batch, n_batch = stopping_para_vae(n_train)

        print("\n" + str(num) + ".", data, "...")
        print(" + Hidden Sizes: ", in_dim, h_sizes, "- Batch_sizes:", batch)
        print(" + Data: %d (%d train, %d vali) - %d normal, %d anomaly" \
              % (len(train_set), n_train, n_vali, \
                 len(test_set[(actual == 1)]), len(test_set[(actual == 0)])))

        print(" + Patience: %5.0d, Validate: %5.0d,  \n + Batch size: %5.0d, n batch:%5.0d" \
              % (pat, val, batch, n_batch))

        AUC_RE = np.empty([0, 10])
        # adadelta, 'adagrad' 'adam''esgd' 'nag''rmsprop' 'rprop' 'sgd'
        # if (num==1):
        sda, re = test_SdA(pre_lr=1e-2,  # re = [stop_ep, vm, tm]
                           end2end_lr=1e-4,
                           algo='adadelta',
                           dataset=datasets,
                           data_name=data,
                           n_validate=n_vali,
                           norm=norm,
                           batch_size=batch,
                           hidden_sizes=h_sizes,
                           corruptions=corruptions,
                           patience=pat,
                           validation=val)

        # Computer AUC on hidden data
        lof, cen, dis, kde, svm05, svm01, ae = sda.Compute_AUC_Hidden(train_X, test_X, actual, norm, data)
        auc_hidden = np.column_stack([batch, re[0], lof, cen, dis, kde, svm05, svm01, ae, 100 * re[2]])
        AUC_Hidden = np.append(AUC_Hidden, auc_hidden)

        # compute standard deviation of z
        # sda.Compute_Std(train_X, test_X, actual, data, path)
        # save hidden data to files
        #        sda.Save_Hidden_Data(train_X, test_X, data, path)

        # store AUC_input AUC_hidden and RE to AUC_RE for each data
        #        AUC_RE   = np.append(AUC_RE, auc_hidden)
        #        AUC_RE   = np.reshape(AUC_RE,(-1,10))
        #
        #        print("\n+ AUC input, AUC hidden:")
        #        np.set_printoptions(precision=3, suppress=True)
        #        column_list = [2,3,4,5,6,7,8,9]
        #        print (AUC_RE[:,column_list])
        #
        #        AUC_Hidden = np.append(AUC_Hidden, auc_hidden)
        AUC_Hidden = np.reshape(AUC_Hidden, (-1, 10))
        np.set_printoptions(precision=3, suppress=True)
        column_list = [2, 3, 4, 5, 6, 7, 8, 9]
        print("    LOF    CEN    MDIS   KDE   SVM5    SVM1    AE    RE*100")
        print(AUC_Hidden[:, column_list])
        print("\n")

    AUC_Hidden = np.reshape(AUC_Hidden, (-1, 10))
    np.set_printoptions(precision=3, suppress=True)
    column_list = [2, 3, 4, 5, 6, 7, 8, 9]
    print("    LOF    CEN    MDIS   KDE   SVM5    SVM1    AE    RE*100")
    print(AUC_Hidden[:, column_list])


#    #store AUC_input and AUC_hidden to AUC_Input, AUC_Hidden
#    AUC_Hidden  =  np.reshape(AUC_Hidden, (-1, 10))
#    np.savetxt(path +  "AUC_Hidden.csv", AUC_Hidden, delimiter=",", fmt='%f' )


if __name__ == '__main__':
    Main_Test()
